In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances  # 直接インポート
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 14, 
    'axes.labelsize': 16, 
    'xtick.labelsize': 14, 
    'ytick.labelsize': 14,
    'legend.fontsize': 12
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    # 1次元配列であることを保証
    idx1 = indices1.flatten()
    idx2 = indices2.flatten()
    
    p1 = atoms.positions[idx1]
    p2 = atoms.positions[idx2]

    # 距離の計算
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    
    # 自己相関排除 (同じ原子ID同士の距離0を除外)
    valid_dists = dists[dists > 0.01] if np.array_equal(idx1, idx2) else dists
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error: {e}"); return

    if len(traj) == 0: return

    # --- 基本情報の取得 ---
    initial_atoms = traj[0] # 初期構造
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H','F']}
    
    # --- 表面/内部 Alの分類 (初期配置のZ座標で判定) ---
    surface_al_indices = []
    bulk_al_indices = []
    if len(indices['Al']) > 0:
        al_positions = initial_atoms.positions[indices['Al']]
        z_max = np.max(al_positions[:, 2])
        # 最表面から 4.0 Angstrom 以内を「表面」と定義
        surface_mask = al_positions[:, 2] > (z_max - 4.0)
        surface_al_indices = indices['Al'][surface_mask]
        bulk_al_indices = indices['Al'][~surface_mask]

    print(f"  -> Total Al: {len(indices['Al'])}, Surface: {len(surface_al_indices)}, Bulk: {len(bulk_al_indices)}")

    # --- データ蓄積用リスト ---
    time_points = []   # 時間 (ps)
    
    # 配位数関連
    cn_avg_list = []      # 平均配位数
    reacted_al_list = []  # 反応した(CN>=1) Alの数
    
    # MSD関連
    msd_total_list = []
    msd_surface_list = []
    msd_bulk_list = []
    
    # 分子生成関連 (H2)
    h2_count_list = []

    # RDF用
    rdf_rmax = 6.0
    rdf_nbins = 100
    rdf_sum = np.zeros(rdf_nbins)
    rdf_count = 0
    start_rdf_frame = int(len(traj) * 0.5)

    # 初期位置の保存 (flattenで1次元化を保証)
    idx_al_total = indices['Al'].flatten()
    idx_al_surface = surface_al_indices.flatten() if len(surface_al_indices) > 0 else np.array([])
    idx_al_bulk = bulk_al_indices.flatten() if len(bulk_al_indices) > 0 else np.array([])
    
    initial_pos_total = initial_atoms.positions[idx_al_total] if len(idx_al_total) > 0 else None
    initial_pos_surface = initial_atoms.positions[idx_al_surface] if len(idx_al_surface) > 0 else None
    initial_pos_bulk = initial_atoms.positions[idx_al_bulk] if len(idx_al_bulk) > 0 else None

    # 時間計算の設定
    # 例: 40000 stepsで40ps。trajに80フレームある場合、1フレーム=0.5ps
    total_sim_time_ps = 40.0 # シミュレーション総時間 (ps) ※適宜変更してください
    dt_ps = total_sim_time_ps / max(len(traj), 1)

    for i, atoms in enumerate(traj):
        current_time = i * dt_ps
        time_points.append(current_time)

        # 1. 配位数解析 (Al-O) - 【修正箇所】
        current_cn_avg = 0
        current_reacted_count = 0
        
        idx_al = indices['Al'].flatten()
        idx_o = indices['O'].flatten()

        if len(idx_al) > 0 and len(idx_o) > 0:
            # ase.geometry.get_distances で計算 (戻り値はフラットな配列)
            p1 = atoms.positions[idx_al]
            p2 = atoms.positions[idx_o]
            _, d_len = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
            
            # (N_Al, N_O) の行列に整形
            d_matrix = d_len.reshape(len(idx_al), len(idx_o))
            
            # 各Al原子ごとのOの数 (2.5A以内)
            cn_per_atom = np.sum(d_matrix < 2.5, axis=1)
            
            current_cn_avg = np.mean(cn_per_atom)
            current_reacted_count = np.sum(cn_per_atom >= 1) # 1つ以上酸素がついているAlの数

        cn_avg_list.append(current_cn_avg)
        reacted_al_list.append(current_reacted_count)

        # 2. MSD計算 (Total, Surface, Bulk)
        def calc_msd(current_atoms_pos, init_pos):
            if init_pos is None or len(init_pos) == 0: return 0
            diff = current_atoms_pos - init_pos
            return np.mean(np.sum(diff**2, axis=1))

        if len(idx_al_total) > 0:
            msd_total_list.append(calc_msd(atoms.positions[idx_al_total], initial_pos_total))
        else:
            msd_total_list.append(0)
            
        if len(idx_al_surface) > 0:
            msd_surface_list.append(calc_msd(atoms.positions[idx_al_surface], initial_pos_surface))
        else:
            msd_surface_list.append(0)

        if len(idx_al_bulk) > 0:
            msd_bulk_list.append(calc_msd(atoms.positions[idx_al_bulk], initial_pos_bulk))
        else:
            msd_bulk_list.append(0)

        # 3. 水素分子(H2)検出
        h2_count = 0
        idx_h = indices['H'].flatten()
        if len(idx_h) > 1:
            p_h = atoms.positions[idx_h]
            _, h_dists = get_distances(p_h, p_h, cell=atoms.cell, pbc=atoms.pbc)
            # 自己距離(0)を除外
            valid_h = h_dists[h_dists > 0.01]
            # ペア数なので2で割る
            h2_count = np.sum(valid_h < 0.85) // 2 
        h2_count_list.append(h2_count)

        # 4. RDF積算
        if i >= start_rdf_frame:
            hist, edges = compute_manual_rdf(atoms, indices['Al'], indices['O'], rmax=rdf_rmax, nbins=rdf_nbins)
            rdf_sum += hist
            rdf_count += 1
            bin_edges = edges

    # --- CSV出力 1: 時系列データ (Trends) ---
    df_trends = pd.DataFrame({
        'Time_ps': time_points,
        'CN_Avg': cn_avg_list,
        'Reacted_Al_Count': reacted_al_list,
        'H2_Count': h2_count_list,
        'MSD_Total': msd_total_list,
        'MSD_Surface': msd_surface_list,
        'MSD_Bulk': msd_bulk_list
    })
    df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)

    # --- グラフ描画 (Time軸) ---
    
    # Plot 1: MSD (Surface vs Bulk)
    plt.figure(figsize=(8, 6))
    plt.plot(time_points, msd_surface_list, label='Surface Al', color='red', lw=2)
    plt.plot(time_points, msd_bulk_list, label='Bulk Al', color='blue', linestyle='--', lw=1)
    plt.plot(time_points, msd_total_list, label='Total Al', color='gray', alpha=0.5)
    plt.xlabel("Time (ps)"); plt.ylabel("MSD ($\AA^2$)")
    plt.title(f"MSD Analysis: {label}")
    plt.legend(); plt.grid(True)
    plt.savefig(output_dir / f"{label}_msd_detailed.png"); plt.close()

    # Plot 2: Reaction Progress (Reacted Count & H2)
    fig, ax1 = plt.subplots(figsize=(8, 6))
    ax2 = ax1.twinx()
    ax1.plot(time_points, reacted_al_list, 'g-', label='Reacted Al Count (CN>=1)', lw=2)
    ax2.plot(time_points, h2_count_list, 'm--', label='H2 Molecule Count', lw=2)
    ax1.set_xlabel("Time (ps)")
    ax1.set_ylabel("Count (Al atoms)", color='g')
    ax2.set_ylabel("Count (H2 molecules)", color='m')
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.title(f"Reaction Progress: {label}")
    plt.savefig(output_dir / f"{label}_reaction.png"); plt.close()

    # Plot 3: Density Profile
    plt.figure(figsize=(8, 5))
    density_data = {}
    bins_z = np.linspace(0, cell_diag[2], 101)
    bin_centers_z = 0.5 * (bins_z[1:] + bins_z[:-1])
    density_data['Z_axis'] = bin_centers_z
    for spec, idx in indices.items():
        idx_flat = idx.flatten()
        if len(idx_flat) > 0:
            hist_vals, _ = np.histogram(final_atoms.positions[idx_flat, 2], bins=bins_z)
            density_data[spec] = hist_vals
            plt.plot(bin_centers_z, hist_vals, label=spec, lw=2)
    pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
    plt.title(f"Density: {label}"); plt.xlabel("Z (Å)"); plt.legend()
    plt.savefig(output_dir / f"{label}_density.png"); plt.close()

    # Plot 4: RDF
    if rdf_count > 0:
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        rho = len(indices['O']) / cell_vol
        shell_vol = 4 * np.pi * r**2 * dr
        gr = (rdf_sum / rdf_count) / (len(indices['Al']) * shell_vol * rho)
        pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)
        plt.figure(figsize=(8, 5))
        plt.plot(r, gr, color='blue', lw=2)
        plt.title(f"RDF Al-O: {label}"); plt.xlabel("r (Å)"); plt.ylabel("g(r)"); plt.grid(True)
        plt.savefig(output_dir / f"{label}_rdf_Al_O.png"); plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_LiOH_v4_NPT")
    analysis_dir = base_dir / "analysis_results"
    analysis_dir.mkdir(parents=True, exist_ok=True)
    
    for target in ["Al_Metal", "Al_Oxide"]:
        for temp in [300, 400, 500, 600, 750, 900]:
            traj_file = base_dir / f"{target}_md_{temp}K.traj"
            if traj_file.exists():
                analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")

if __name__ == "__main__":
    main_analysis()

Analyzing: Al_Metal_md_300K.traj
  -> Total Al: 996, Surface: 76, Bulk: 920


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances  # 直接インポート
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 14, 
    'axes.labelsize': 16, 
    'xtick.labelsize': 14, 
    'ytick.labelsize': 14,
    'legend.fontsize': 12
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    # 1次元配列であることを保証
    idx1 = indices1.flatten()
    idx2 = indices2.flatten()
    
    p1 = atoms.positions[idx1]
    p2 = atoms.positions[idx2]

    # 距離の計算
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    
    # 自己相関排除 (同じ原子ID同士の距離0を除外)
    valid_dists = dists[dists > 0.01] if np.array_equal(idx1, idx2) else dists
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error: {e}"); return

    if len(traj) == 0: return

    # --- 基本情報の取得 ---
    initial_atoms = traj[0] # 初期構造
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H','F']}
    
    # --- 表面/内部 Alの分類 (初期配置のZ座標で判定) ---
    surface_al_indices = []
    bulk_al_indices = []
    if len(indices['Al']) > 0:
        al_positions = initial_atoms.positions[indices['Al']]
        z_max = np.max(al_positions[:, 2])
        # 最表面から 4.0 Angstrom 以内を「表面」と定義
        surface_mask = al_positions[:, 2] > (z_max - 4.0)
        surface_al_indices = indices['Al'][surface_mask]
        bulk_al_indices = indices['Al'][~surface_mask]

    print(f"  -> Total Al: {len(indices['Al'])}, Surface: {len(surface_al_indices)}, Bulk: {len(bulk_al_indices)}")

    # --- データ蓄積用リスト ---
    time_points = []   # 時間 (ps)
    
    # 配位数関連
    cn_avg_list = []      # 平均配位数
    reacted_al_list = []  # 反応した(CN>=1) Alの数
    
    # MSD関連
    msd_total_list = []
    msd_surface_list = []
    msd_bulk_list = []
    
    # 分子生成関連 (H2)
    h2_count_list = []

    # RDF用
    rdf_rmax = 6.0
    rdf_nbins = 100
    rdf_sum = np.zeros(rdf_nbins)
    rdf_count = 0
    start_rdf_frame = int(len(traj) * 0.5)

    # 初期位置の保存 (flattenで1次元化を保証)
    idx_al_total = indices['Al'].flatten()
    idx_al_surface = surface_al_indices.flatten() if len(surface_al_indices) > 0 else np.array([])
    idx_al_bulk = bulk_al_indices.flatten() if len(bulk_al_indices) > 0 else np.array([])
    
    initial_pos_total = initial_atoms.positions[idx_al_total] if len(idx_al_total) > 0 else None
    initial_pos_surface = initial_atoms.positions[idx_al_surface] if len(idx_al_surface) > 0 else None
    initial_pos_bulk = initial_atoms.positions[idx_al_bulk] if len(idx_al_bulk) > 0 else None

    # 時間計算の設定
    # 例: 40000 stepsで40ps。trajに80フレームある場合、1フレーム=0.5ps
    total_sim_time_ps = 40.0 # シミュレーション総時間 (ps) ※適宜変更してください
    dt_ps = total_sim_time_ps / max(len(traj), 1)

    for i, atoms in enumerate(traj):
        current_time = i * dt_ps
        time_points.append(current_time)

        # 1. 配位数解析 (Al-O) - 【修正箇所】
        current_cn_avg = 0
        current_reacted_count = 0
        
        idx_al = indices['Al'].flatten()
        idx_o = indices['O'].flatten()

        if len(idx_al) > 0 and len(idx_o) > 0:
            # ase.geometry.get_distances で計算 (戻り値はフラットな配列)
            p1 = atoms.positions[idx_al]
            p2 = atoms.positions[idx_o]
            _, d_len = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
            
            # (N_Al, N_O) の行列に整形
            d_matrix = d_len.reshape(len(idx_al), len(idx_o))
            
            # 各Al原子ごとのOの数 (2.5A以内)
            cn_per_atom = np.sum(d_matrix < 2.5, axis=1)
            
            current_cn_avg = np.mean(cn_per_atom)
            current_reacted_count = np.sum(cn_per_atom >= 1) # 1つ以上酸素がついているAlの数

        cn_avg_list.append(current_cn_avg)
        reacted_al_list.append(current_reacted_count)

        # 2. MSD計算 (Total, Surface, Bulk)
        def calc_msd(current_atoms_pos, init_pos):
            if init_pos is None or len(init_pos) == 0: return 0
            diff = current_atoms_pos - init_pos
            return np.mean(np.sum(diff**2, axis=1))

        if len(idx_al_total) > 0:
            msd_total_list.append(calc_msd(atoms.positions[idx_al_total], initial_pos_total))
        else:
            msd_total_list.append(0)
            
        if len(idx_al_surface) > 0:
            msd_surface_list.append(calc_msd(atoms.positions[idx_al_surface], initial_pos_surface))
        else:
            msd_surface_list.append(0)

        if len(idx_al_bulk) > 0:
            msd_bulk_list.append(calc_msd(atoms.positions[idx_al_bulk], initial_pos_bulk))
        else:
            msd_bulk_list.append(0)

        # 3. 水素分子(H2)検出
        h2_count = 0
        idx_h = indices['H'].flatten()
        if len(idx_h) > 1:
            p_h = atoms.positions[idx_h]
            _, h_dists = get_distances(p_h, p_h, cell=atoms.cell, pbc=atoms.pbc)
            # 自己距離(0)を除外
            valid_h = h_dists[h_dists > 0.01]
            # ペア数なので2で割る
            h2_count = np.sum(valid_h < 0.85) // 2 
        h2_count_list.append(h2_count)

        # 4. RDF積算
        if i >= start_rdf_frame:
            hist, edges = compute_manual_rdf(atoms, indices['Al'], indices['O'], rmax=rdf_rmax, nbins=rdf_nbins)
            rdf_sum += hist
            rdf_count += 1
            bin_edges = edges

    # --- CSV出力 1: 時系列データ (Trends) ---
    df_trends = pd.DataFrame({
        'Time_ps': time_points,
        'CN_Avg': cn_avg_list,
        'Reacted_Al_Count': reacted_al_list,
        'H2_Count': h2_count_list,
        'MSD_Total': msd_total_list,
        'MSD_Surface': msd_surface_list,
        'MSD_Bulk': msd_bulk_list
    })
    df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)

    # --- グラフ描画 (Time軸) ---
    
    # Plot 1: MSD (Surface vs Bulk)
    plt.figure(figsize=(8, 6))
    plt.plot(time_points, msd_surface_list, label='Surface Al', color='red', lw=2)
    plt.plot(time_points, msd_bulk_list, label='Bulk Al', color='blue', linestyle='--', lw=1)
    plt.plot(time_points, msd_total_list, label='Total Al', color='gray', alpha=0.5)
    plt.xlabel("Time (ps)"); plt.ylabel("MSD ($\AA^2$)")
    plt.title(f"MSD Analysis: {label}")
    plt.legend(); plt.grid(True)
    plt.savefig(output_dir / f"{label}_msd_detailed.png"); plt.close()

    # Plot 2: Reaction Progress (Reacted Count & H2)
    fig, ax1 = plt.subplots(figsize=(8, 6))
    ax2 = ax1.twinx()
    ax1.plot(time_points, reacted_al_list, 'g-', label='Reacted Al Count (CN>=1)', lw=2)
    ax2.plot(time_points, h2_count_list, 'm--', label='H2 Molecule Count', lw=2)
    ax1.set_xlabel("Time (ps)")
    ax1.set_ylabel("Count (Al atoms)", color='g')
    ax2.set_ylabel("Count (H2 molecules)", color='m')
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.title(f"Reaction Progress: {label}")
    plt.savefig(output_dir / f"{label}_reaction.png"); plt.close()

    # Plot 3: Density Profile
    plt.figure(figsize=(8, 5))
    density_data = {}
    bins_z = np.linspace(0, cell_diag[2], 101)
    bin_centers_z = 0.5 * (bins_z[1:] + bins_z[:-1])
    density_data['Z_axis'] = bin_centers_z
    for spec, idx in indices.items():
        idx_flat = idx.flatten()
        if len(idx_flat) > 0:
            hist_vals, _ = np.histogram(final_atoms.positions[idx_flat, 2], bins=bins_z)
            density_data[spec] = hist_vals
            plt.plot(bin_centers_z, hist_vals, label=spec, lw=2)
    pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
    plt.title(f"Density: {label}"); plt.xlabel("Z (Å)"); plt.legend()
    plt.savefig(output_dir / f"{label}_density.png"); plt.close()

    # Plot 4: RDF
    if rdf_count > 0:
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        rho = len(indices['O']) / cell_vol
        shell_vol = 4 * np.pi * r**2 * dr
        gr = (rdf_sum / rdf_count) / (len(indices['Al']) * shell_vol * rho)
        pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)
        plt.figure(figsize=(8, 5))
        plt.plot(r, gr, color='blue', lw=2)
        plt.title(f"RDF Al-O: {label}"); plt.xlabel("r (Å)"); plt.ylabel("g(r)"); plt.grid(True)
        plt.savefig(output_dir / f"{label}_rdf_Al_O.png"); plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_Li_F")
    analysis_dir = base_dir / "analysis_results"
    analysis_dir.mkdir(parents=True, exist_ok=True)
    
    for target in ["Al_Metal", "Al_Oxide"]:
        for temp in [300, 400, 500, 600, 750, 900]:
            traj_file = base_dir / f"{target}_md_{temp}K.traj"
            if traj_file.exists():
                analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")

if __name__ == "__main__":
    main_analysis()

Analyzing: Al_Metal_md_300K.traj
  -> Total Al: 996, Surface: 89, Bulk: 907


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances  # 直接インポート
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 20, 
    'axes.labelsize': 20, 
    'xtick.labelsize': 20, 
    'ytick.labelsize': 20,
    'legend.fontsize': 20
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    # 1次元配列であることを保証
    idx1 = indices1.flatten()
    idx2 = indices2.flatten()
    
    p1 = atoms.positions[idx1]
    p2 = atoms.positions[idx2]

    # 距離の計算
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    
    # 自己相関排除 (同じ原子ID同士の距離0を除外)
    valid_dists = dists[dists > 0.01] if np.array_equal(idx1, idx2) else dists
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error: {e}"); return

    if len(traj) == 0: return

    # --- 基本情報の取得 ---
    initial_atoms = traj[0] # 初期構造
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H','F']}
    
    # --- 表面/内部 Alの分類 (初期配置のZ座標で判定) ---
    surface_al_indices = []
    bulk_al_indices = []
    if len(indices['Al']) > 0:
        al_positions = initial_atoms.positions[indices['Al']]
        z_max = np.max(al_positions[:, 2])
        # 最表面から 4.0 Angstrom 以内を「表面」と定義
        surface_mask = al_positions[:, 2] > (z_max - 4.0)
        surface_al_indices = indices['Al'][surface_mask]
        bulk_al_indices = indices['Al'][~surface_mask]

    print(f"  -> Total Al: {len(indices['Al'])}, Surface: {len(surface_al_indices)}, Bulk: {len(bulk_al_indices)}")

    # --- データ蓄積用リスト ---
    time_points = []   # 時間 (ps)
    
    # 配位数関連
    cn_avg_list = []      # 平均配位数
    reacted_al_list = []  # 反応した(CN>=1) Alの数
    
    # MSD関連
    msd_total_list = []
    msd_surface_list = []
    msd_bulk_list = []
    
    # 分子生成関連 (H2)
    h2_count_list = []

    # RDF用
    rdf_rmax = 6.0
    rdf_nbins = 100
    rdf_sum = np.zeros(rdf_nbins)
    rdf_count = 0
    start_rdf_frame = int(len(traj) * 0.5)

    # 初期位置の保存 (flattenで1次元化を保証)
    idx_al_total = indices['Al'].flatten()
    idx_al_surface = surface_al_indices.flatten() if len(surface_al_indices) > 0 else np.array([])
    idx_al_bulk = bulk_al_indices.flatten() if len(bulk_al_indices) > 0 else np.array([])
    
    initial_pos_total = initial_atoms.positions[idx_al_total] if len(idx_al_total) > 0 else None
    initial_pos_surface = initial_atoms.positions[idx_al_surface] if len(idx_al_surface) > 0 else None
    initial_pos_bulk = initial_atoms.positions[idx_al_bulk] if len(idx_al_bulk) > 0 else None

    # 時間計算の設定
    # 例: 40000 stepsで40ps。trajに80フレームある場合、1フレーム=0.5ps
    total_sim_time_ps = 40.0 # シミュレーション総時間 (ps) ※適宜変更してください
    dt_ps = total_sim_time_ps / max(len(traj), 1)

    for i, atoms in enumerate(traj):
        current_time = i * dt_ps
        time_points.append(current_time)

        # 1. 配位数解析 (Al-O) - 【修正箇所】
        current_cn_avg = 0
        current_reacted_count = 0
        
        idx_al = indices['Al'].flatten()
        idx_o = indices['O'].flatten()

        if len(idx_al) > 0 and len(idx_o) > 0:
            # ase.geometry.get_distances で計算 (戻り値はフラットな配列)
            p1 = atoms.positions[idx_al]
            p2 = atoms.positions[idx_o]
            _, d_len = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
            
            # (N_Al, N_O) の行列に整形
            d_matrix = d_len.reshape(len(idx_al), len(idx_o))
            
            # 各Al原子ごとのOの数 (2.5A以内)
            cn_per_atom = np.sum(d_matrix < 2.5, axis=1)
            
            current_cn_avg = np.mean(cn_per_atom)
            current_reacted_count = np.sum(cn_per_atom >= 1) # 1つ以上酸素がついているAlの数

        cn_avg_list.append(current_cn_avg)
        reacted_al_list.append(current_reacted_count)

        # 2. MSD計算 (Total, Surface, Bulk)
        def calc_msd(current_atoms_pos, init_pos):
            if init_pos is None or len(init_pos) == 0: return 0
            diff = current_atoms_pos - init_pos
            return np.mean(np.sum(diff**2, axis=1))

        if len(idx_al_total) > 0:
            msd_total_list.append(calc_msd(atoms.positions[idx_al_total], initial_pos_total))
        else:
            msd_total_list.append(0)
            
        if len(idx_al_surface) > 0:
            msd_surface_list.append(calc_msd(atoms.positions[idx_al_surface], initial_pos_surface))
        else:
            msd_surface_list.append(0)

        if len(idx_al_bulk) > 0:
            msd_bulk_list.append(calc_msd(atoms.positions[idx_al_bulk], initial_pos_bulk))
        else:
            msd_bulk_list.append(0)

        # 3. 水素分子(H2)検出
        h2_count = 0
        idx_h = indices['H'].flatten()
        if len(idx_h) > 1:
            p_h = atoms.positions[idx_h]
            _, h_dists = get_distances(p_h, p_h, cell=atoms.cell, pbc=atoms.pbc)
            # 自己距離(0)を除外
            valid_h = h_dists[h_dists > 0.01]
            # ペア数なので2で割る
            h2_count = np.sum(valid_h < 0.85) // 2 
        h2_count_list.append(h2_count)

        # 4. RDF積算
        if i >= start_rdf_frame:
            hist, edges = compute_manual_rdf(atoms, indices['Al'], indices['O'], rmax=rdf_rmax, nbins=rdf_nbins)
            rdf_sum += hist
            rdf_count += 1
            bin_edges = edges

    # --- CSV出力 1: 時系列データ (Trends) ---
    df_trends = pd.DataFrame({
        'Time_ps': time_points,
        'CN_Avg': cn_avg_list,
        'Reacted_Al_Count': reacted_al_list,
        'H2_Count': h2_count_list,
        'MSD_Total': msd_total_list,
        'MSD_Surface': msd_surface_list,
        'MSD_Bulk': msd_bulk_list
    })
    df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)

    # --- グラフ描画 (Time軸) ---
    
    # Plot 1: MSD (Surface vs Bulk)
    plt.figure(figsize=(8, 6))
    plt.plot(time_points, msd_surface_list, label='Surface Al', color='red', lw=2)
    plt.plot(time_points, msd_bulk_list, label='Bulk Al', color='blue', linestyle='--', lw=1)
    plt.plot(time_points, msd_total_list, label='Total Al', color='gray', alpha=0.5)
    plt.xlabel("Time (ps)"); plt.ylabel("MSD ($\AA^2$)")
    plt.title(f"MSD Analysis: {label}")
    plt.legend(); plt.grid(True)
    plt.savefig(output_dir / f"{label}_msd_detailed.png"); plt.close()

    # Plot 2: Reaction Progress (Reacted Count & H2)
    fig, ax1 = plt.subplots(figsize=(8, 6))
    ax2 = ax1.twinx()
    ax1.plot(time_points, reacted_al_list, 'g-', label='Reacted Al Count (CN>=1)', lw=2)
    ax2.plot(time_points, h2_count_list, 'm--', label='H2 Molecule Count', lw=2)
    ax1.set_xlabel("Time (ps)")
    ax1.set_ylabel("Count (Al atoms)", color='g')
    ax2.set_ylabel("Count (H2 molecules)", color='m')
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.title(f"Reaction Progress: {label}")
    plt.savefig(output_dir / f"{label}_reaction.png"); plt.close()

    # Plot 3: Density Profile
    plt.figure(figsize=(8, 5))
    density_data = {}
    bins_z = np.linspace(0, cell_diag[2], 101)
    bin_centers_z = 0.5 * (bins_z[1:] + bins_z[:-1])
    density_data['Z_axis'] = bin_centers_z
    for spec, idx in indices.items():
        idx_flat = idx.flatten()
        if len(idx_flat) > 0:
            hist_vals, _ = np.histogram(final_atoms.positions[idx_flat, 2], bins=bins_z)
            density_data[spec] = hist_vals
            plt.plot(bin_centers_z, hist_vals, label=spec, lw=2)
    pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
    plt.title(f"Density: {label}"); plt.xlabel("Z (Å)"); plt.legend()
    plt.savefig(output_dir / f"{label}_density.png"); plt.close()

    # Plot 4: RDF
    if rdf_count > 0:
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        rho = len(indices['O']) / cell_vol
        shell_vol = 4 * np.pi * r**2 * dr
        gr = (rdf_sum / rdf_count) / (len(indices['Al']) * shell_vol * rho)
        pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)
        plt.figure(figsize=(8, 5))
        plt.plot(r, gr, color='blue', lw=2)
        plt.title(f"RDF Al-O: {label}"); plt.xlabel("r (Å)"); plt.ylabel("g(r)"); plt.grid(True)
        plt.savefig(output_dir / f"{label}_rdf_Al_O.png"); plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_Na_F_OH")
    analysis_dir = base_dir / "analysis_results"
    analysis_dir.mkdir(parents=True, exist_ok=True)
    
    for target in [ "Al_Oxide"]:
        for temp in [300, 400, 500, 600, 750, 900]:
            traj_file = base_dir / f"{target}_md_{temp}K.traj"
            if traj_file.exists():
                analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")

if __name__ == "__main__":
    main_analysis()

Analyzing: Al_Oxide_md_300K.traj
  -> Total Al: 608, Surface: 52, Bulk: 556
   -> Finished Al_Oxide_300K
Analyzing: Al_Oxide_md_400K.traj
  -> Total Al: 608, Surface: 55, Bulk: 553
   -> Finished Al_Oxide_400K
Analyzing: Al_Oxide_md_500K.traj
  -> Total Al: 608, Surface: 55, Bulk: 553
   -> Finished Al_Oxide_500K
Analyzing: Al_Oxide_md_600K.traj
  -> Total Al: 608, Surface: 38, Bulk: 570
   -> Finished Al_Oxide_600K
Analyzing: Al_Oxide_md_750K.traj
  -> Total Al: 608, Surface: 39, Bulk: 569
   -> Finished Al_Oxide_750K
Analyzing: Al_Oxide_md_900K.traj
  -> Total Al: 608, Surface: 44, Bulk: 564
   -> Finished Al_Oxide_900K


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances  # 直接インポート
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 14, 
    'axes.labelsize': 16, 
    'xtick.labelsize': 14, 
    'ytick.labelsize': 14,
    'legend.fontsize': 12
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    # 1次元配列であることを保証
    idx1 = indices1.flatten()
    idx2 = indices2.flatten()
    
    p1 = atoms.positions[idx1]
    p2 = atoms.positions[idx2]

    # 距離の計算
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    
    # 自己相関排除 (同じ原子ID同士の距離0を除外)
    valid_dists = dists[dists > 0.01] if np.array_equal(idx1, idx2) else dists
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error: {e}"); return

    if len(traj) == 0: return

    # --- 基本情報の取得 ---
    initial_atoms = traj[0] # 初期構造
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H','F']}
    
    # --- 表面/内部 Alの分類 (初期配置のZ座標で判定) ---
    surface_al_indices = []
    bulk_al_indices = []
    if len(indices['Al']) > 0:
        al_positions = initial_atoms.positions[indices['Al']]
        z_max = np.max(al_positions[:, 2])
        # 最表面から 4.0 Angstrom 以内を「表面」と定義
        surface_mask = al_positions[:, 2] > (z_max - 4.0)
        surface_al_indices = indices['Al'][surface_mask]
        bulk_al_indices = indices['Al'][~surface_mask]

    print(f"  -> Total Al: {len(indices['Al'])}, Surface: {len(surface_al_indices)}, Bulk: {len(bulk_al_indices)}")

    # --- データ蓄積用リスト ---
    time_points = []   # 時間 (ps)
    
    # 配位数関連
    cn_avg_list = []      # 平均配位数
    reacted_al_list = []  # 反応した(CN>=1) Alの数
    
    # MSD関連
    msd_total_list = []
    msd_surface_list = []
    msd_bulk_list = []
    
    # 分子生成関連 (H2)
    h2_count_list = []

    # RDF用
    rdf_rmax = 6.0
    rdf_nbins = 100
    rdf_sum = np.zeros(rdf_nbins)
    rdf_count = 0
    start_rdf_frame = int(len(traj) * 0.5)

    # 初期位置の保存 (flattenで1次元化を保証)
    idx_al_total = indices['Al'].flatten()
    idx_al_surface = surface_al_indices.flatten() if len(surface_al_indices) > 0 else np.array([])
    idx_al_bulk = bulk_al_indices.flatten() if len(bulk_al_indices) > 0 else np.array([])
    
    initial_pos_total = initial_atoms.positions[idx_al_total] if len(idx_al_total) > 0 else None
    initial_pos_surface = initial_atoms.positions[idx_al_surface] if len(idx_al_surface) > 0 else None
    initial_pos_bulk = initial_atoms.positions[idx_al_bulk] if len(idx_al_bulk) > 0 else None

    # 時間計算の設定
    # 例: 40000 stepsで40ps。trajに80フレームある場合、1フレーム=0.5ps
    total_sim_time_ps = 40.0 # シミュレーション総時間 (ps) ※適宜変更してください
    dt_ps = total_sim_time_ps / max(len(traj), 1)

    for i, atoms in enumerate(traj):
        current_time = i * dt_ps
        time_points.append(current_time)

        # 1. 配位数解析 (Al-O) - 【修正箇所】
        current_cn_avg = 0
        current_reacted_count = 0
        
        idx_al = indices['Al'].flatten()
        idx_o = indices['O'].flatten()

        if len(idx_al) > 0 and len(idx_o) > 0:
            # ase.geometry.get_distances で計算 (戻り値はフラットな配列)
            p1 = atoms.positions[idx_al]
            p2 = atoms.positions[idx_o]
            _, d_len = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
            
            # (N_Al, N_O) の行列に整形
            d_matrix = d_len.reshape(len(idx_al), len(idx_o))
            
            # 各Al原子ごとのOの数 (2.5A以内)
            cn_per_atom = np.sum(d_matrix < 2.5, axis=1)
            
            current_cn_avg = np.mean(cn_per_atom)
            current_reacted_count = np.sum(cn_per_atom >= 1) # 1つ以上酸素がついているAlの数

        cn_avg_list.append(current_cn_avg)
        reacted_al_list.append(current_reacted_count)

        # 2. MSD計算 (Total, Surface, Bulk)
        def calc_msd(current_atoms_pos, init_pos):
            if init_pos is None or len(init_pos) == 0: return 0
            diff = current_atoms_pos - init_pos
            return np.mean(np.sum(diff**2, axis=1))

        if len(idx_al_total) > 0:
            msd_total_list.append(calc_msd(atoms.positions[idx_al_total], initial_pos_total))
        else:
            msd_total_list.append(0)
            
        if len(idx_al_surface) > 0:
            msd_surface_list.append(calc_msd(atoms.positions[idx_al_surface], initial_pos_surface))
        else:
            msd_surface_list.append(0)

        if len(idx_al_bulk) > 0:
            msd_bulk_list.append(calc_msd(atoms.positions[idx_al_bulk], initial_pos_bulk))
        else:
            msd_bulk_list.append(0)

        # 3. 水素分子(H2)検出
        h2_count = 0
        idx_h = indices['H'].flatten()
        if len(idx_h) > 1:
            p_h = atoms.positions[idx_h]
            _, h_dists = get_distances(p_h, p_h, cell=atoms.cell, pbc=atoms.pbc)
            # 自己距離(0)を除外
            valid_h = h_dists[h_dists > 0.01]
            # ペア数なので2で割る
            h2_count = np.sum(valid_h < 0.85) // 2 
        h2_count_list.append(h2_count)

        # 4. RDF積算
        if i >= start_rdf_frame:
            hist, edges = compute_manual_rdf(atoms, indices['Al'], indices['O'], rmax=rdf_rmax, nbins=rdf_nbins)
            rdf_sum += hist
            rdf_count += 1
            bin_edges = edges

    # --- CSV出力 1: 時系列データ (Trends) ---
    df_trends = pd.DataFrame({
        'Time_ps': time_points,
        'CN_Avg': cn_avg_list,
        'Reacted_Al_Count': reacted_al_list,
        'H2_Count': h2_count_list,
        'MSD_Total': msd_total_list,
        'MSD_Surface': msd_surface_list,
        'MSD_Bulk': msd_bulk_list
    })
    df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)

    # --- グラフ描画 (Time軸) ---
    
    # Plot 1: MSD (Surface vs Bulk)
    plt.figure(figsize=(8, 6))
    plt.plot(time_points, msd_surface_list, label='Surface Al', color='red', lw=2)
    plt.plot(time_points, msd_bulk_list, label='Bulk Al', color='blue', linestyle='--', lw=1)
    plt.plot(time_points, msd_total_list, label='Total Al', color='gray', alpha=0.5)
    plt.xlabel("Time (ps)"); plt.ylabel("MSD ($\AA^2$)")
    plt.title(f"MSD Analysis: {label}")
    plt.legend(); plt.grid(True)
    plt.savefig(output_dir / f"{label}_msd_detailed.png"); plt.close()

    # Plot 2: Reaction Progress (Reacted Count & H2)
    fig, ax1 = plt.subplots(figsize=(8, 6))
    ax2 = ax1.twinx()
    ax1.plot(time_points, reacted_al_list, 'g-', label='Reacted Al Count (CN>=1)', lw=2)
    ax2.plot(time_points, h2_count_list, 'm--', label='H2 Molecule Count', lw=2)
    ax1.set_xlabel("Time (ps)")
    ax1.set_ylabel("Count (Al atoms)", color='g')
    ax2.set_ylabel("Count (H2 molecules)", color='m')
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.title(f"Reaction Progress: {label}")
    plt.savefig(output_dir / f"{label}_reaction.png"); plt.close()

    # Plot 3: Density Profile
    plt.figure(figsize=(8, 5))
    density_data = {}
    bins_z = np.linspace(0, cell_diag[2], 101)
    bin_centers_z = 0.5 * (bins_z[1:] + bins_z[:-1])
    density_data['Z_axis'] = bin_centers_z
    for spec, idx in indices.items():
        idx_flat = idx.flatten()
        if len(idx_flat) > 0:
            hist_vals, _ = np.histogram(final_atoms.positions[idx_flat, 2], bins=bins_z)
            density_data[spec] = hist_vals
            plt.plot(bin_centers_z, hist_vals, label=spec, lw=2)
    pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
    plt.title(f"Density: {label}"); plt.xlabel("Z (Å)"); plt.legend()
    plt.savefig(output_dir / f"{label}_density.png"); plt.close()

    # Plot 4: RDF
    if rdf_count > 0:
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        rho = len(indices['O']) / cell_vol
        shell_vol = 4 * np.pi * r**2 * dr
        gr = (rdf_sum / rdf_count) / (len(indices['Al']) * shell_vol * rho)
        pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)
        plt.figure(figsize=(8, 5))
        plt.plot(r, gr, color='blue', lw=2)
        plt.title(f"RDF Al-O: {label}"); plt.xlabel("r (Å)"); plt.ylabel("g(r)"); plt.grid(True)
        plt.savefig(output_dir / f"{label}_rdf_Al_O.png"); plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_LiOH_v8")
    analysis_dir = base_dir / "analysis_results"
    analysis_dir.mkdir(parents=True, exist_ok=True)
    
    for target in ["Al_Metal", "Al_Oxide"]:
        for temp in [300, 400, 500, 600, 750, 900]:
            traj_file = base_dir / f"{target}_md_{temp}K.traj"
            if traj_file.exists():
                analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")

if __name__ == "__main__":
    main_analysis()

<>:196: SyntaxWarning: invalid escape sequence '\A'
<>:196: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipykernel_26002/2771888322.py:196: SyntaxWarning: invalid escape sequence '\A'
  plt.xlabel("Time (ps)"); plt.ylabel("MSD ($\AA^2$)")


Analyzing: Al_Metal_md_300K.traj
  -> Total Al: 996, Surface: 130, Bulk: 866
   -> Finished Al_Metal_300K
Analyzing: Al_Metal_md_400K.traj
  -> Total Al: 996, Surface: 118, Bulk: 878
   -> Finished Al_Metal_400K
Analyzing: Al_Metal_md_500K.traj
  -> Total Al: 996, Surface: 92, Bulk: 904
   -> Finished Al_Metal_500K
Analyzing: Al_Oxide_md_300K.traj
  -> Total Al: 608, Surface: 65, Bulk: 543
   -> Finished Al_Oxide_300K
Analyzing: Al_Oxide_md_400K.traj
  -> Total Al: 608, Surface: 60, Bulk: 548
   -> Finished Al_Oxide_400K
Analyzing: Al_Oxide_md_500K.traj
  -> Total Al: 608, Surface: 62, Bulk: 546
   -> Finished Al_Oxide_500K


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances  # 直接インポート
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 14, 
    'axes.labelsize': 16, 
    'xtick.labelsize': 14, 
    'ytick.labelsize': 14,
    'legend.fontsize': 12
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    # 1次元配列であることを保証
    idx1 = indices1.flatten()
    idx2 = indices2.flatten()
    
    p1 = atoms.positions[idx1]
    p2 = atoms.positions[idx2]

    # 距離の計算
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    
    # 自己相関排除 (同じ原子ID同士の距離0を除外)
    valid_dists = dists[dists > 0.01] if np.array_equal(idx1, idx2) else dists
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error: {e}"); return

    if len(traj) == 0: return

    # --- 基本情報の取得 ---
    initial_atoms = traj[0] # 初期構造
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H','F']}
    
    # --- 表面/内部 Alの分類 (初期配置のZ座標で判定) ---
    surface_al_indices = []
    bulk_al_indices = []
    if len(indices['Al']) > 0:
        al_positions = initial_atoms.positions[indices['Al']]
        z_max = np.max(al_positions[:, 2])
        # 最表面から 4.0 Angstrom 以内を「表面」と定義
        surface_mask = al_positions[:, 2] > (z_max - 4.0)
        surface_al_indices = indices['Al'][surface_mask]
        bulk_al_indices = indices['Al'][~surface_mask]

    print(f"  -> Total Al: {len(indices['Al'])}, Surface: {len(surface_al_indices)}, Bulk: {len(bulk_al_indices)}")

    # --- データ蓄積用リスト ---
    time_points = []   # 時間 (ps)
    
    # 配位数関連
    cn_avg_list = []      # 平均配位数
    reacted_al_list = []  # 反応した(CN>=1) Alの数
    
    # MSD関連
    msd_total_list = []
    msd_surface_list = []
    msd_bulk_list = []
    
    # 分子生成関連 (H2)
    h2_count_list = []

    # RDF用
    rdf_rmax = 6.0
    rdf_nbins = 100
    rdf_sum = np.zeros(rdf_nbins)
    rdf_count = 0
    start_rdf_frame = int(len(traj) * 0.5)

    # 初期位置の保存 (flattenで1次元化を保証)
    idx_al_total = indices['Al'].flatten()
    idx_al_surface = surface_al_indices.flatten() if len(surface_al_indices) > 0 else np.array([])
    idx_al_bulk = bulk_al_indices.flatten() if len(bulk_al_indices) > 0 else np.array([])
    
    initial_pos_total = initial_atoms.positions[idx_al_total] if len(idx_al_total) > 0 else None
    initial_pos_surface = initial_atoms.positions[idx_al_surface] if len(idx_al_surface) > 0 else None
    initial_pos_bulk = initial_atoms.positions[idx_al_bulk] if len(idx_al_bulk) > 0 else None

    # 時間計算の設定
    # 例: 40000 stepsで40ps。trajに80フレームある場合、1フレーム=0.5ps
    total_sim_time_ps = 40.0 # シミュレーション総時間 (ps) ※適宜変更してください
    dt_ps = total_sim_time_ps / max(len(traj), 1)

    for i, atoms in enumerate(traj):
        current_time = i * dt_ps
        time_points.append(current_time)

        # 1. 配位数解析 (Al-O) - 【修正箇所】
        current_cn_avg = 0
        current_reacted_count = 0
        
        idx_al = indices['Al'].flatten()
        idx_o = indices['O'].flatten()

        if len(idx_al) > 0 and len(idx_o) > 0:
            # ase.geometry.get_distances で計算 (戻り値はフラットな配列)
            p1 = atoms.positions[idx_al]
            p2 = atoms.positions[idx_o]
            _, d_len = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
            
            # (N_Al, N_O) の行列に整形
            d_matrix = d_len.reshape(len(idx_al), len(idx_o))
            
            # 各Al原子ごとのOの数 (2.5A以内)
            cn_per_atom = np.sum(d_matrix < 2.5, axis=1)
            
            current_cn_avg = np.mean(cn_per_atom)
            current_reacted_count = np.sum(cn_per_atom >= 1) # 1つ以上酸素がついているAlの数

        cn_avg_list.append(current_cn_avg)
        reacted_al_list.append(current_reacted_count)

        # 2. MSD計算 (Total, Surface, Bulk)
        def calc_msd(current_atoms_pos, init_pos):
            if init_pos is None or len(init_pos) == 0: return 0
            diff = current_atoms_pos - init_pos
            return np.mean(np.sum(diff**2, axis=1))

        if len(idx_al_total) > 0:
            msd_total_list.append(calc_msd(atoms.positions[idx_al_total], initial_pos_total))
        else:
            msd_total_list.append(0)
            
        if len(idx_al_surface) > 0:
            msd_surface_list.append(calc_msd(atoms.positions[idx_al_surface], initial_pos_surface))
        else:
            msd_surface_list.append(0)

        if len(idx_al_bulk) > 0:
            msd_bulk_list.append(calc_msd(atoms.positions[idx_al_bulk], initial_pos_bulk))
        else:
            msd_bulk_list.append(0)

        # 3. 水素分子(H2)検出
        h2_count = 0
        idx_h = indices['H'].flatten()
        if len(idx_h) > 1:
            p_h = atoms.positions[idx_h]
            _, h_dists = get_distances(p_h, p_h, cell=atoms.cell, pbc=atoms.pbc)
            # 自己距離(0)を除外
            valid_h = h_dists[h_dists > 0.01]
            # ペア数なので2で割る
            h2_count = np.sum(valid_h < 0.85) // 2 
        h2_count_list.append(h2_count)

        # 4. RDF積算
        if i >= start_rdf_frame:
            hist, edges = compute_manual_rdf(atoms, indices['Al'], indices['O'], rmax=rdf_rmax, nbins=rdf_nbins)
            rdf_sum += hist
            rdf_count += 1
            bin_edges = edges

    # --- CSV出力 1: 時系列データ (Trends) ---
    df_trends = pd.DataFrame({
        'Time_ps': time_points,
        'CN_Avg': cn_avg_list,
        'Reacted_Al_Count': reacted_al_list,
        'H2_Count': h2_count_list,
        'MSD_Total': msd_total_list,
        'MSD_Surface': msd_surface_list,
        'MSD_Bulk': msd_bulk_list
    })
    df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)

    # --- グラフ描画 (Time軸) ---
    
    # Plot 1: MSD (Surface vs Bulk)
    plt.figure(figsize=(8, 6))
    plt.plot(time_points, msd_surface_list, label='Surface Al', color='red', lw=2)
    plt.plot(time_points, msd_bulk_list, label='Bulk Al', color='blue', linestyle='--', lw=1)
    plt.plot(time_points, msd_total_list, label='Total Al', color='gray', alpha=0.5)
    plt.xlabel("Time (ps)"); plt.ylabel("MSD ($\AA^2$)")
    plt.title(f"MSD Analysis: {label}")
    plt.legend(); plt.grid(True)
    plt.savefig(output_dir / f"{label}_msd_detailed.png"); plt.close()

    # Plot 2: Reaction Progress (Reacted Count & H2)
    fig, ax1 = plt.subplots(figsize=(8, 6))
    ax2 = ax1.twinx()
    ax1.plot(time_points, reacted_al_list, 'g-', label='Reacted Al Count (CN>=1)', lw=2)
    ax2.plot(time_points, h2_count_list, 'm--', label='H2 Molecule Count', lw=2)
    ax1.set_xlabel("Time (ps)")
    ax1.set_ylabel("Count (Al atoms)", color='g')
    ax2.set_ylabel("Count (H2 molecules)", color='m')
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    plt.title(f"Reaction Progress: {label}")
    plt.savefig(output_dir / f"{label}_reaction.png"); plt.close()

    # Plot 3: Density Profile
    plt.figure(figsize=(8, 5))
    density_data = {}
    bins_z = np.linspace(0, cell_diag[2], 101)
    bin_centers_z = 0.5 * (bins_z[1:] + bins_z[:-1])
    density_data['Z_axis'] = bin_centers_z
    for spec, idx in indices.items():
        idx_flat = idx.flatten()
        if len(idx_flat) > 0:
            hist_vals, _ = np.histogram(final_atoms.positions[idx_flat, 2], bins=bins_z)
            density_data[spec] = hist_vals
            plt.plot(bin_centers_z, hist_vals, label=spec, lw=2)
    pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
    plt.title(f"Density: {label}"); plt.xlabel("Z (Å)"); plt.legend()
    plt.savefig(output_dir / f"{label}_density.png"); plt.close()

    # Plot 4: RDF
    if rdf_count > 0:
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        rho = len(indices['O']) / cell_vol
        shell_vol = 4 * np.pi * r**2 * dr
        gr = (rdf_sum / rdf_count) / (len(indices['Al']) * shell_vol * rho)
        pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)
        plt.figure(figsize=(8, 5))
        plt.plot(r, gr, color='blue', lw=2)
        plt.title(f"RDF Al-O: {label}"); plt.xlabel("r (Å)"); plt.ylabel("g(r)"); plt.grid(True)
        plt.savefig(output_dir / f"{label}_rdf_Al_O.png"); plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    files=["Al_LiOH_v8","Al_Li_F_OH","Al_Li_F","Al_Na_F_OH"]
    for file in files:
        base_dir = Path(f"/home/jovyan/Kaori/MD/LiB_2/structure/{file}")
        analysis_dir = base_dir / "analysis_results"
        analysis_dir.mkdir(parents=True, exist_ok=True)
        
        for target in ["Al_Metal", "Al_Oxide"]:
            for temp in [300, 400, 500, 600, 750, 900]:
                traj_file = base_dir / f"{target}_md_{temp}K.traj"
                if traj_file.exists():
                    analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")

if __name__ == "__main__":
    main_analysis()

Analyzing: Al_Metal_md_300K.traj
  -> Total Al: 996, Surface: 93, Bulk: 903
   -> Finished Al_Metal_300K
Analyzing: Al_Metal_md_400K.traj
  -> Total Al: 996, Surface: 74, Bulk: 922
   -> Finished Al_Metal_400K
Analyzing: Al_Metal_md_500K.traj
  -> Total Al: 996, Surface: 51, Bulk: 945
   -> Finished Al_Metal_500K
Analyzing: Al_Metal_md_600K.traj
  -> Total Al: 996, Surface: 52, Bulk: 944
   -> Finished Al_Metal_600K
Analyzing: Al_Metal_md_750K.traj
  -> Total Al: 996, Surface: 46, Bulk: 950
   -> Finished Al_Metal_750K
Analyzing: Al_Metal_md_900K.traj
  -> Total Al: 996, Surface: 2, Bulk: 994
   -> Finished Al_Metal_900K


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances
from pathlib import Path
import warnings
import math

warnings.filterwarnings('ignore')

# --- 1. フォントサイズ等の設定 (22pt) ---
plt.rcParams.update({
    'font.size': 22, 
    'axes.labelsize': 22, 
    'xtick.labelsize': 22, 
    'ytick.labelsize': 22,
    'legend.fontsize': 18,
    'figure.titlesize': 24
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    idx1 = indices1.flatten()
    idx2 = indices2.flatten()
    
    p1 = atoms.positions[idx1]
    p2 = atoms.positions[idx2]

    # 距離計算
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    
    # 自己相関排除 (同じ原子ID同士の距離0を除外)
    if np.array_equal(idx1, idx2):
        valid_dists = dists[dists > 0.01]
    else:
        valid_dists = dists.flatten()
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error reading traj: {e}"); return

    if len(traj) == 0: return

    # --- 基本設定 ---
    initial_atoms = traj[0]
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    # 元素ごとのインデックス取得
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H', 'F']}

    # --- 解析ペアの設定 ---
    # name: 表示名, ele1/ele2: 元素記号, cutoff: 結合判定距離(Å)
    target_pairs = [
        {'name': 'Al-F', 'ele1': 'Al', 'ele2': 'F', 'cutoff': 2.0},
        {'name': 'Li-F', 'ele1': 'Li', 'ele2': 'F', 'cutoff': 2.0},
        {'name': 'O-H',  'ele1': 'O',  'ele2': 'H', 'cutoff': 1.25}, # 水酸基等の共有結合
        {'name': 'Al-O', 'ele1': 'Al', 'ele2': 'O', 'cutoff': 2.0}  # 既存
    ]

    # データ格納用辞書の初期化
    results = {
        'time': [],
        'cn_data': {p['name']: [] for p in target_pairs}, # 平均配位数
        'rdf_sum': {p['name']: np.zeros(100) for p in target_pairs},
        'rdf_count': 0
    }
    
    # RDF用パラメータ
    rdf_rmax = 6.0
    rdf_nbins = 100
    start_rdf_frame = int(len(traj) * 0.5)
    
    # 時間設定 (例: 1step=1fsなど。ここでは仮に全40psとする)
    total_sim_time_ps = 40.0 
    dt_ps = total_sim_time_ps / max(len(traj), 1)

    # --- メインループ ---
    for i, atoms in enumerate(traj):
        current_time = i * dt_ps
        results['time'].append(current_time)

        # 各ペアごとの結合数(CN)計算
        for pair in target_pairs:
            idx1 = indices.get(pair['ele1'], np.array([])).flatten()
            idx2 = indices.get(pair['ele2'], np.array([])).flatten()
            
            cn_avg = 0
            if len(idx1) > 0 and len(idx2) > 0:
                p1 = atoms.positions[idx1]
                p2 = atoms.positions[idx2]
                _, d_len = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
                
                # 行列化 (N_ele1 x N_ele2)
                d_matrix = d_len.reshape(len(idx1), len(idx2))
                
                # 同種原子の場合、対角成分(自分自身)を除外するための処理が必要だが、
                # ここでは異種原子ペア(Al-F等)メインのため簡易実装
                # カットオフ以内の数をカウント
                cn_per_atom = np.sum(d_matrix < pair['cutoff'], axis=1)
                
                # ele1(例:Al) 1原子あたりの ele2(例:F) の平均数
                cn_avg = np.mean(cn_per_atom)
            
            results['cn_data'][pair['name']].append(cn_avg)

        # RDF積算 (後半フレームのみ)
        if i >= start_rdf_frame:
            results['rdf_count'] += 1
            for pair in target_pairs:
                idx1 = indices.get(pair['ele1'], np.array([]))
                idx2 = indices.get(pair['ele2'], np.array([]))
                hist, edges = compute_manual_rdf(atoms, idx1, idx2, rmax=rdf_rmax, nbins=rdf_nbins)
                results['rdf_sum'][pair['name']] += hist
                bin_edges = edges # すべて同じビン設定なので上書きでOK

    # --- CSV出力 (時系列データ) ---
    df_data = {'Time_ps': results['time']}
    for name, data in results['cn_data'].items():
        df_data[f'CN_Avg_{name}'] = data
    pd.DataFrame(df_data).to_csv(output_dir / f"{label}_bond_counts.csv", index=False)


    # --- グラフ作成 1: 結合数 (Bond Counts) ---
    # レイアウト: 2列 x N行
    n_plots = len(target_pairs)
    n_cols = 2
    n_rows = math.ceil(n_plots / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6 * n_rows), constrained_layout=True)
    axes_flat = axes.flatten() if n_plots > 1 else [axes]

    for j, pair in enumerate(target_pairs):
        ax = axes_flat[j]
        name = pair['name']
        data = results['cn_data'][name]
        time = results['time']
        
        ax.plot(time, data, lw=3, color='tab:blue')
        ax.set_title(f"{name} Bond Count (cut={pair['cutoff']}A)")
        ax.set_xlabel("Time (ps)")
        ax.set_ylabel("Avg CN")
        ax.grid(True)

    # 余ったサブプロットを消す
    for k in range(j+1, len(axes_flat)):
        axes_flat[k].axis('off')
        
    plt.suptitle(f"Bond Counts Trends: {label}")
    plt.savefig(output_dir / f"{label}_bonds_summary.png")
    plt.close()


    # --- グラフ作成 2: RDF ---
    if results['rdf_count'] > 0:
        fig_rdf, axes_rdf = plt.subplots(n_rows, n_cols, figsize=(16, 6 * n_rows), constrained_layout=True)
        axes_rdf_flat = axes_rdf.flatten() if n_plots > 1 else [axes_rdf]
        
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        
        rdf_df_data = {'r': r}

        for j, pair in enumerate(target_pairs):
            ax = axes_rdf_flat[j]
            name = pair['name']
            
            idx1_len = len(indices.get(pair['ele1'], []))
            idx2_len = len(indices.get(pair['ele2'], []))
            
            if idx1_len > 0 and idx2_len > 0:
                # 密度計算 (ele2の数密度)
                rho = idx2_len / cell_vol
                shell_vol = 4 * np.pi * r**2 * dr
                
                # g(r) = (ヒストグラム平均) / (中心原子数 * シェル体積 * 密度)
                gr = (results['rdf_sum'][name] / results['rdf_count']) / (idx1_len * shell_vol * rho)
                
                rdf_df_data[f'g_r_{name}'] = gr
                
                ax.plot(r, gr, lw=3, color='tab:red')
                ax.set_title(f"RDF: {name}")
                ax.set_xlabel("r (Å)")
                ax.set_ylabel("g(r)")
                ax.grid(True)
            else:
                ax.text(0.5, 0.5, "No Data", ha='center')

        # 余ったサブプロットを消す
        for k in range(j+1, len(axes_rdf_flat)):
            axes_rdf_flat[k].axis('off')

        pd.DataFrame(rdf_df_data).to_csv(output_dir / f"{label}_rdf_summary.csv", index=False)
        plt.suptitle(f"RDF Summary: {label}")
        plt.savefig(output_dir / f"{label}_rdf_summary.png")
        plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_Li_F_OH")
    analysis_dir = base_dir / "analysis_results"
    analysis_dir.mkdir(parents=True, exist_ok=True)
    
    # 対象ファイル名パターン (必要に応じて変更してください)
    # 例: Al_Metal_md_300K.traj など
    targets = ["Al_Metal", "Al_Oxide"]
    temps = [300, 400, 500, 600, 750, 900]

    for target in targets:
        for temp in temps:
            traj_file = base_dir / f"{target}_md_{temp}K.traj"
            if traj_file.exists():
                analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")
            else:
                print(f"Skipping (not found): {traj_file.name}")

if __name__ == "__main__":
    main_analysis()

Analyzing: Al_Metal_md_300K.traj
   -> Finished Al_Metal_300K
Analyzing: Al_Metal_md_400K.traj
   -> Finished Al_Metal_400K
Analyzing: Al_Metal_md_500K.traj
   -> Finished Al_Metal_500K
Analyzing: Al_Metal_md_600K.traj
   -> Finished Al_Metal_600K
Analyzing: Al_Metal_md_750K.traj
   -> Finished Al_Metal_750K
Analyzing: Al_Metal_md_900K.traj
   -> Finished Al_Metal_900K
Analyzing: Al_Oxide_md_300K.traj
   -> Finished Al_Oxide_300K
Analyzing: Al_Oxide_md_400K.traj
   -> Finished Al_Oxide_400K
Analyzing: Al_Oxide_md_500K.traj
   -> Finished Al_Oxide_500K
Analyzing: Al_Oxide_md_600K.traj
   -> Finished Al_Oxide_600K
Analyzing: Al_Oxide_md_750K.traj
   -> Finished Al_Oxide_750K
Analyzing: Al_Oxide_md_900K.traj
   -> Finished Al_Oxide_900K


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import math
import numpy as np

def format_face_label(face):
    """ラベルを3桁に揃える (例: 1 -> 001)"""
    s = str(face)
    if s.isdigit():
        return s.zfill(3)
    return s

def plot_nmc_integrated(csv_dir="analysis_csv", output_dir="plots"):
    csv_dir, output_dir = Path(csv_dir), Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    def load_and_prepare(filename):
        path = csv_dir / filename
        if not path.exists(): return None
        df = pd.read_csv(path)
        if 'Face' in df.columns:
            df['Face'] = df['Face'].apply(format_face_label)
        return df

    # フォント設定
    plt.rcParams.update({'font.size': 10, 'axes.labelsize': 11, 'axes.titlesize': 12})

    # 1. 反応トレンド (同じ結晶面を、異なる温度で並べる)
    df_t = load_and_prepare("summary_trends.csv")
    if df_t is not None:
        print("📊 トレンド解析中...")
        faces = sorted(df_t['Face'].unique())
        for face in faces:
            data_face = df_t[df_t['Face'] == face]
            temps = sorted(data_face['Temp'].unique(), key=lambda x: float(re.sub(r'\D', '', str(x))) if any(c.isdigit() for c in str(x)) else str(x))
            
            rows = math.ceil(len(temps) / 2)
            fig, axes = plt.subplots(rows, 2, figsize=(12, 4 * rows), squeeze=False)
            
            for i, temp in enumerate(temps):
                ax = axes.flatten()[i]
                d = data_face[data_face['Temp'] == temp].sort_values('Frame')
                ax.plot(d['Frame'], d['Leached_Li'], label='Leached Li', color='#1f77b4', lw=2)
                ax.plot(d['Frame'], d['Intruded_H'], label='Intruded H', color='#d62728', lw=2)
                
                ax.set_title(f"Face: {face} | Temp: {temp}K")
                ax.set_xlabel("Time (Frame)")
                ax.set_ylabel("Atom Count")
                ax.legend(loc='upper left')
                ax.grid(True, alpha=0.3)
            
            for j in range(i + 1, len(axes.flatten())): axes.flatten()[j].axis('off')
            plt.tight_layout()
            plt.savefig(output_dir / f"grid_trend_face_{face}.png", dpi=300)
            plt.close()

    # 2. 密度プロファイル (同じ結晶面を、異なる温度で並べる)
    df_d = load_and_prepare("summary_density.csv")
    if df_d is not None:
        print("🌡️ 密度プロファイル解析中...")
        faces = sorted(df_d['Face'].unique())
        for face in faces:
            data_face = df_d[df_d['Face'] == face]
            temps = sorted(data_face['Temp'].unique(), key=lambda x: float(re.sub(r'\D', '', str(x))) if any(c.isdigit() for c in str(x)) else str(x))
            
            rows = math.ceil(len(temps) / 2)
            fig, axes = plt.subplots(rows, 2, figsize=(12, 4 * rows), squeeze=False)
            
            for i, temp in enumerate(temps):
                ax = axes.flatten()[i]
                d = data_face[data_face['Temp'] == temp].sort_values('Z')
                
                colors = {'Li': '#1f77b4', 'H': '#d62728', 'O': '#7f7f7f'}
                for s in ['Li', 'H', 'O']:
                    ds = d[(d['Symbol'] == s) & (d['Density'] > 1e-4)]
                    if not ds.empty:
                        ax.fill_between(ds['Z'], ds['Density'], alpha=0.3, color=colors[s], label=s)
                        ax.plot(ds['Z'], ds['Density'], color=colors[s], lw=1.5)
                
                ax.set_title(f"Density Profile: {face} ({temp}K)")
                ax.set_xlabel("Z coordinate (Å)")
                ax.set_ylabel("Atomic Density")
                ax.legend(loc='upper right')
                ax.grid(axis='y', alpha=0.2)
                
            for j in range(i + 1, len(axes.flatten())): axes.flatten()[j].axis('off')
            plt.tight_layout()
            plt.savefig(output_dir / f"grid_density_face_{face}.png", dpi=300)
            plt.close()

    # 3. RDF (同じ結晶面を、異なる温度で並べる)
    df_r = load_and_prepare("summary_rdf.csv")
    if df_r is not None:
        print("🔍 RDF解析中...")
        faces = sorted(df_r['Face'].unique())
        for face in faces:
            data_face = df_r[df_r['Face'] == face]
            temps = sorted(data_face['Temp'].unique(), key=lambda x: float(re.sub(r'\D', '', str(x))) if any(c.isdigit() for c in str(x)) else str(x))
            
            rows = math.ceil(len(temps) / 2)
            fig, axes = plt.subplots(rows, 2, figsize=(12, 4 * rows), squeeze=False)
            
            for i, temp in enumerate(temps):
                ax = axes.flatten()[i]
                d = data_face[data_face['Temp'] == temp]
                
                for p, c in [('Li-LatticeO', '#1f77b4'), ('Li-WaterO', '#ff7f0e')]:
                    dp = d[d['Pair'] == p].sort_values('r')
                    if not dp.empty: ax.plot(dp['r'], dp['g_r'], label=p, color=c, lw=2)
                
                ax.axvline(2.1, color='gray', ls='--', alpha=0.6, label='2.1Å (Bond)')
                ax.axvline(2.8, color='green', ls=':', alpha=0.6, label='2.8Å (Contact)')
                
                ax.set_title(f"RDF: {face} ({temp}K)")
                ax.set_xlabel("Distance r (Å)")
                ax.set_ylabel("g(r)")
                ax.set_xlim(1.0, 5.0)
                ax.set_ylim(0, None)
                ax.legend()
                ax.grid(True, alpha=0.2)
                
            for j in range(i + 1, len(axes.flatten())): axes.flatten()[j].axis('off')
            plt.tight_layout()
            plt.savefig(output_dir / f"grid_rdf_face_{face}.png", dpi=300)
            plt.close()

import re # 必要に応じて追加

if __name__ == "__main__":
    plot_nmc_integrated()

📊 トレンド解析中...
🌡️ 密度プロファイル解析中...
🔍 RDF解析中...


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd  # 追加
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 14, 
    'axes.labelsize': 16, 
    'xtick.labelsize': 14, 
    'ytick.labelsize': 14,
    'legend.fontsize': 12
})

# --- RDFを手動で計算する関数 ---
def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    
    # 距離の計算 (PBC考慮)
    dists = get_distances(atoms.positions[indices1], atoms.positions[indices2], 
                          cell=atoms.cell, pbc=atoms.pbc)[1]
    
    # 自己相関排除
    valid_dists = dists[dists > 0.01] if np.array_equal(indices1, indices2) else dists.flatten()
    
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label):
    print(f"Analyzing: {traj_path.name}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error: {e}"); return

    if len(traj) == 0: return

    # 基本情報の取得
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H','F']}
    
    # --- データ蓄積用 ---
    rdf_rmax = 6.0
    rdf_nbins = 100
    rdf_sum = np.zeros(rdf_nbins)
    rdf_count = 0
    
    # 時系列データ用リスト
    time_steps = []
    cn_list = []
    msd_al_list = []
    
    initial_al_pos = traj[0].positions[indices['Al']]
    
    # 後半50%のフレームでRDFを平均化
    start_rdf_frame = int(len(traj) * 0.5)

    for i, atoms in enumerate(traj):
        time_steps.append(i* 0.5) # ステップ数（または時間）
        
        # 1. 配位数計算 (Al-O, cutoff 2.5A)
        cn_val = 0
        if len(indices['Al']) > 0 and len(indices['O']) > 0:
            d = get_distances(atoms.positions[indices['Al']], atoms.positions[indices['O']], 
                              cell=atoms.cell, pbc=atoms.pbc)[1]
            cn_val = np.sum(d < 2.5) / len(indices['Al'])
        cn_list.append(cn_val)

        # 2. MSD計算 (Al)
        msd_val = 0
        if len(indices['Al']) > 0:
            diff = atoms.positions[indices['Al']] - initial_al_pos
            # PBC補正が必要な場合は別途処理が必要だが、ここでは簡易的に絶対座標変位とみなすか、unwrapped trajectory推奨
            # 今回は元のコード準拠で計算
            msd_val = np.mean(np.sum(diff**2, axis=1))
        msd_al_list.append(msd_val)

        # 3. RDF積算
        if i >= start_rdf_frame:
            hist, edges = compute_manual_rdf(atoms, indices['Al'], indices['O'], rmax=rdf_rmax, nbins=rdf_nbins)
            rdf_sum += hist
            rdf_count += 1
            bin_edges = edges

    # --- CSV出力 1: 時系列データ (Trends) ---
    df_trends = pd.DataFrame({
        'Step': time_steps,
        'CN_Al_O': cn_list,
        'MSD_Al': msd_al_list
    })
    df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)


    # --- グラフ保存 & CSV出力 2: 密度プロファイル ---
    plt.figure(figsize=(8, 5))
    density_data = {}
    bins_z = np.linspace(0, cell_diag[2], 101) # ビン設定
    bin_centers_z = 0.5 * (bins_z[1:] + bins_z[:-1])
    density_data['Z_axis'] = bin_centers_z

    for spec, idx in indices.items():
        if len(idx) > 0:
            # ヒストグラム計算
            hist_vals, _ = np.histogram(final_atoms.positions[idx, 2], bins=bins_z)
            density_data[spec] = hist_vals
            
            # プロット
            plt.plot(bin_centers_z, hist_vals, label=spec, lw=2)
            
    # Density CSV保存
    pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
    
    plt.title(f"Density: {label}"); plt.xlabel("Z (Å)"); plt.legend()
    plt.savefig(output_dir / f"{label}_density.png"); plt.close()


    # --- グラフ保存 & CSV出力 3: RDF ---
    if rdf_count > 0:
        r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
        dr = bin_edges[1] - bin_edges[0]
        rho = len(indices['O']) / cell_vol
        shell_vol = 4 * np.pi * r**2 * dr
        
        # g(r) 計算
        gr = (rdf_sum / rdf_count) / (len(indices['Al']) * shell_vol * rho)
        
        # RDF CSV保存
        pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)

        plt.figure(figsize=(8, 5))
        plt.plot(r, gr, color='blue', lw=2)
        plt.title(f"RDF Al-O: {label}"); plt.xlabel("r (Å)"); plt.ylabel("g(r)"); plt.grid(True)
        plt.savefig(output_dir / f"{label}_rdf_Al_O.png"); plt.close()

    # (3) 配位数とMSD (グラフのみ)
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()
    ax1.plot(cn_list, 'g-', label='Coord. Number')
    ax2.plot(msd_al_list, 'r-', label='Al-MSD')
    ax1.set_ylabel("CN (Al-O)", color='g'); ax2.set_ylabel("MSD (Å^2)", color='r')
    plt.title(f"Trends: {label}"); plt.savefig(output_dir / f"{label}_trends.png"); plt.close()

    print(f"   -> Finished {label}")

def main_analysis():
    base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_LiOH_v4")
    analysis_dir = base_dir / "analysis_results"
    analysis_dir.mkdir(parents=True, exist_ok=True)
    for target in ["Al_Metal", "Al_Oxide"]:
        for temp in [300, 400, 500, 600, 750, 900]:
            traj_file = base_dir / f"{target}_md_{temp}K.traj"
            if traj_file.exists():
                analyze_md_results(traj_file, analysis_dir, f"{target}_{temp}K")

if __name__ == "__main__":
    main_analysis()

Analyzing: Al_Metal_md_300K.traj


In [3]:
import pandas as pd
from pathlib import Path
import warnings

def merge_csv_files(target_dir, file_pattern="*.csv", output_name="merged_output.csv"):
    """
    指定ディレクトリ内のCSVを結合する関数
    
    Args:
        target_dir (str or Path): 対象のフォルダパス
        file_pattern (str): 読み込むファイルのパターン（例: "*trends.csv"）
        output_name (str): 出力するファイル名
    """
    dir_path = Path(target_dir)
    
    # マッチするファイルを探す
    all_files = list(dir_path.glob(file_pattern))
    
    # 出力ファイル自体がすでにある場合、読み込み対象から除外する
    all_files = [f for f in all_files if f.name != output_name]
    
    if not all_files:
        print(f"No files found matching '{file_pattern}' in {dir_path}")
        return

    print(f"Found {len(all_files)} files. Merging...")

    df_list = []
    
    for file in all_files:
        try:
            # CSVを読み込む
            df = pd.read_csv(file)
            
            # どのファイルから来たデータかわかるように列を追加
            # 例: "Al_Metal_300K_trends.csv" -> "Al_Metal_300K_trends"
            df.insert(0, 'Source_File', file.stem)
            
            # ファイル名からターゲットと温度を抽出（必要であれば）
            # ファイル名が "Al_Metal_300K_..." のような形式を想定
            parts = file.stem.split('_')
            if len(parts) >= 3:
                # 簡易的な抽出ロジック（ファイル名形式に合わせて調整可）
                # 例: Target=Al_Metal, Temp=300K
                target_guess = f"{parts[0]}_{parts[1]}"
                temp_guess = parts[2]
                df.insert(1, 'Target', target_guess)
                df.insert(2, 'Temp', temp_guess)
            
            df_list.append(df)
            
        except Exception as e:
            print(f"Error reading {file.name}: {e}")

    # 全データを結合
    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
        
        # 保存
        output_path = dir_path / output_name
        merged_df.to_csv(output_path, index=False)
        print(f"Successfully saved merged CSV to: {output_path}")
        print(f"Total rows: {len(merged_df)}")
    else:
        print("No valid data to merge.")

def main():
    # === 設定部分 ===
    files=["Al_LiOH_v8","Al_Li_F_OH","Al_Li_F","Al_Na_F_OH"]
    for file in files:
        base_dir = Path(f"/home/jovyan/Kaori/MD/LiB_2/structure/{file}/analysis_results")
        # base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Al_Li_F/analysis_results")
    
        # 1. トレンドデータ（MSD, CNの時系列）をまとめる場合
        print("--- Merging Trends ---")
        merge_csv_files(base_dir, file_pattern="*trends.csv", output_name="all_trends_summary.csv")
        
        # 2. 密度プロファイルをまとめる場合
        print("\n--- Merging Density Profiles ---")
        merge_csv_files(base_dir, file_pattern="*density.csv", output_name="all_density_summary.csv")
        
        # 3. RDFをまとめる場合
        print("\n--- Merging RDF Data ---")
        merge_csv_files(base_dir, file_pattern="*rdf_Al_O.csv", output_name="all_rdf_summary.csv")

if __name__ == "__main__":
    main()

--- Merging Trends ---
Found 6 files. Merging...
Successfully saved merged CSV to: /home/jovyan/Kaori/MD/LiB_2/structure/Al_LiOH_v8/analysis_results/all_trends_summary.csv
Total rows: 408

--- Merging Density Profiles ---
Found 6 files. Merging...
Successfully saved merged CSV to: /home/jovyan/Kaori/MD/LiB_2/structure/Al_LiOH_v8/analysis_results/all_density_summary.csv
Total rows: 600

--- Merging RDF Data ---
Found 6 files. Merging...
Successfully saved merged CSV to: /home/jovyan/Kaori/MD/LiB_2/structure/Al_LiOH_v8/analysis_results/all_rdf_summary.csv
Total rows: 600
--- Merging Trends ---
Found 12 files. Merging...
Successfully saved merged CSV to: /home/jovyan/Kaori/MD/LiB_2/structure/Al_Li_F_OH/analysis_results/all_trends_summary.csv
Total rows: 960

--- Merging Density Profiles ---
Found 12 files. Merging...
Successfully saved merged CSV to: /home/jovyan/Kaori/MD/LiB_2/structure/Al_Li_F_OH/analysis_results/all_density_summary.csv
Total rows: 1200

--- Merging RDF Data ---
Found 1

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from pathlib import Path

# --- 設定 ---
files = ["Al_LiOH_v8", "Al_Li_F_OH", "Al_Li_F", "Al_Na_F_OH"]
base_path = Path("/home/jovyan/Kaori/MD/LiB_2/structure")
output_base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Activation_Energy_Results")
output_base_dir.mkdir(parents=True, exist_ok=True)

R = 8.314  # 気体定数 [J/(mol*K)]
t_target = 39.5
temp_min_limit = 600  # 活性化エネルギー算出に使う最低温度

def calculate_ea(df, target_name='Al_Metal', t_target=39.5, temp_min=600):
    """
    アレニウスプロットから活性化エネルギーを算出する
    """
    # 1. ターゲット（Al_Metal等）でフィルタリング
    df_target = df[df['Target'] == target_name].copy()
    if df_target.empty:
        return pd.DataFrame()

    # 2. Temp列のクリーンアップ ('K' 除去)
    if df_target['Temp'].dtype == object:
        df_target['Temp'] = df_target['Temp'].astype(str).str.replace('K', '', regex=False).astype(float)

    # 3. 各温度ごとに target_time (39.5ps) に最も近い MSD を抽出
    extracted_points = []
    unique_temps = sorted(df_target['Temp'].unique())
    
    for temp in unique_temps:
        temp_subset = df_target[df_target['Temp'] == temp]
        
        # Time_ps が存在する列であることを確認
        if 'Time_ps' not in temp_subset.columns:
            continue
            
        # target_time に最も近い行を選択
        idx = (temp_subset['Time_ps'] - t_target).abs().idxmin()
        row = temp_subset.loc[idx]
        
        # 許容誤差範囲内(1.0ps)であれば採用
        if abs(row['Time_ps'] - t_target) <= 1.0:
            extracted_points.append({
                'Temp': temp,
                'MSD_at_t': row['MSD_Surface'] # 大文字小文字はCSVに合わせる
            })
    
    analysis_df = pd.DataFrame(extracted_points)
    if analysis_df.empty:
        return pd.DataFrame()

    # 4. 活性化エネルギーの回帰分析 (高温域のみ)
    subset = analysis_df[analysis_df['Temp'] >= temp_min].dropna()
    
    results = []
    if len(subset) >= 2:
        inv_t = 1.0 / subset['Temp']      # 1/T
        ln_msd = np.log(subset['MSD_at_t']) # ln(MSD)
        
        # 線形回帰
        slope, intercept, r_value, p_value, std_err = stats.linregress(inv_t, ln_msd)
        
        # Ea [J/mol] = -slope * R
        ea_kj_mol = (-slope * R) / 1000.0
        
        results.append({
            'Condition': file, # 外側のループの変数
            'Target': target_name,
            'Ea [kJ/mol]': round(ea_kj_mol, 2),
            'R-squared': round(r_value**2, 4),
            'Temp_Range': f"{int(subset['Temp'].min())}-{int(subset['Temp'].max())}K",
            'Time_ps': t_target
        })
                
    return pd.DataFrame(results)

# --- メインループ ---
for file in files:
    input_file = base_path / file / "analysis_results" / "all_trends_summary.csv"
    output_file = output_base_dir / f"{file}.csv"
    
    if not input_file.exists():
        # もし trends_summary に Time_ps がない場合は、all_trends.csv 等を確認
        input_file = base_path / file / "analysis_results" / "all_trends.csv"
        if not input_file.exists():
            print(f"Skip: {file} のデータが見つかりませんでした。")
            continue
    
    print(f"Processing: {file}...")
    df = pd.read_csv(input_file)
    
    # 実行
    result_df = calculate_ea(df, target_name='Al_Metal', t_target=t_target, temp_min=temp_min_limit)
    
    if not result_df.empty:
        print("--- 活性化エネルギー算出結果 ---")
        print(result_df)
        result_df.to_csv(output_file, index=False)
        print(f"結果を {output_file} に保存しました。\n")
    else:
        print(f"{file}: 計算可能なデータが不足しています。\n")

Processing: Al_LiOH_v8...
Al_LiOH_v8: 計算可能なデータが不足しています。

Processing: Al_Li_F_OH...
Al_Li_F_OH: 計算可能なデータが不足しています。

Processing: Al_Li_F...
--- 活性化エネルギー算出結果 ---
  Condition    Target  Ea [kJ/mol]  R-squared Temp_Range  Time_ps
0   Al_Li_F  Al_Metal         8.42     0.0756   600-900K     39.5
結果を /home/jovyan/Kaori/MD/LiB_2/structure/Activation_Energy_Results/Al_Li_F.csv に保存しました。

Processing: Al_Na_F_OH...
--- 活性化エネルギー算出結果 ---
    Condition    Target  Ea [kJ/mol]  R-squared Temp_Range  Time_ps
0  Al_Na_F_OH  Al_Metal        38.46     0.8682   600-900K     39.5
結果を /home/jovyan/Kaori/MD/LiB_2/structure/Activation_Energy_Results/Al_Na_F_OH.csv に保存しました。



In [8]:
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

# --- 設定 ---
files = ["Al_Li_F_OH"]
 # "Al_Li_F", "Al_Na_F_OH","Al_LiOH_v8", ]
base_path = Path("/home/jovyan/Kaori/MD/LiB_2/structure")
output_base_dir = Path("/home/jovyan/Kaori/MD/LiB_2/structure/Activation_Energy_Results")
output_base_dir.mkdir(parents=True, exist_ok=True)

R = 8.314
t_target = 39.5

# 検討したい温度帯のリスト (min, max)
temp_ranges = [
    (300, 400),
    (300, 500),# 全範囲
    (600, 900), # 高温域のみ（前回）
    (750, 900)  # 超高温域（格子崩壊が顕著な領域）
]

def analyze_ea_variations(df, file_name, t_target=39.5, target_name='Al_Metal'):
    # 1. 前処理
    df_target = df[df['Target'] == target_name].copy()
    if df_target['Temp'].dtype == object:
        df_target['Temp'] = df_target['Temp'].astype(str).str.replace('K', '', regex=False).astype(float)

    # 2. 39.5psのデータを抽出
    extracted = []
    for temp in sorted(df_target['Temp'].unique()):
        temp_subset = df_target[df_target['Temp'] == temp]
        if 'Time_ps' not in temp_subset.columns: continue
        
        idx = (temp_subset['Time_ps'] - t_target).abs().idxmin()
        row = temp_subset.loc[idx]
        if abs(row['Time_ps'] - t_target) <= 1.0:
            extracted.append({'Temp': temp, 'MSD': row['MSD_Surface']})
    
    analysis_df = pd.DataFrame(extracted)
    if analysis_df.empty: return pd.DataFrame()

    results = []
    # 3. 各温度帯で計算
    for t_min, t_max in temp_ranges:
        subset = analysis_df[(analysis_df['Temp'] >= t_min) & (analysis_df['Temp'] <= t_max)].dropna()
        
        if len(subset) >= 2:
            x = 1.0 / subset['Temp']
            y = np.log(subset['MSD'])
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
            
            results.append({
                'Condition': file_name,
                'Range': f"{t_min}-{t_max}K",
                'Ea [kJ/mol]': round((-slope * R) / 1000.0, 2),
                'R-squared': round(r_value**2, 4),
                'Points': len(subset)
            })
    
    return pd.DataFrame(results)

# --- 実行 ---
all_comparison = []
for file in files:
    input_file = base_path / file / "analysis_results" / "all_trends_summary.csv"
    if not input_file.exists():
        input_file = base_path / file / "analysis_results" / "all_trends.csv"
        if not input_file.exists(): continue
    
    df = pd.read_csv(input_file)
    res = analyze_ea_variations(df, file)
    if not res.empty:
        print(f"--- Analysis for {file} ---")
        print(res)
        all_comparison.append(res)

# 全データの比較表を保存
if all_comparison:
    summary_df = pd.concat(all_comparison)
    summary_df.to_csv(output_base_dir / "Ea_Range_Comparison.csv", index=False)

In [9]:
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

# --- 設定 ---
files = ["Al_Li_F_OH"]
base_path = Path("/home/jovyan/Kaori/MD/LiB_2/structure")
output_base_dir = base_path / "Activation_Energy_Results"
output_base_dir.mkdir(parents=True, exist_ok=True)

R = 8.314
t_target = 39.5

# 検討したい温度帯のリスト
temp_ranges = [
    (300, 400),
    (300, 500),
    (600, 900),
    (750, 900)
]

def analyze_ea_variations(df, file_name, t_target=39.5, target_name='Al_Metal'):
    print(f"\n--- Debugging {file_name} ---")
    
    # 1. ターゲットフィルタリングのチェック
    df_target = df[df['Target'] == target_name].copy()
    print(f"1. Target '{target_name}' filtered: {len(df_target)} rows found.")
    if df_target.empty:
        print(f"   [!] Available Targets are: {df['Target'].unique()}")
        return pd.DataFrame()

    # 2. 温度のクリーンアップ
    if df_target['Temp'].dtype == object:
        df_target['Temp'] = df_target['Temp'].astype(str).str.replace('K', '', regex=False).astype(float)
    
    available_temps = sorted(df_target['Temp'].unique())
    print(f"2. Available Temperatures: {available_temps}")

    # 3. カラム名の確認
    msd_col = None
    for col in ['MSD_Surface', 'MSD_surface', 'msd_surface']:
        if col in df_target.columns:
            msd_col = col
            break
    if not msd_col:
        print(f"   [!] Error: MSD column not found. Columns: {df_target.columns.tolist()}")
        return pd.DataFrame()
    print(f"3. Using MSD column: '{msd_col}'")

    # 4. 39.5psのデータを抽出
    extracted = []
    print(f"4. Extracting data near {t_target} ps...")
    for temp in available_temps:
        temp_subset = df_target[df_target['Temp'] == temp]
        
        if 'Time_ps' not in temp_subset.columns:
            print(f"   [!] 'Time_ps' not found for Temp {temp}K")
            continue
            
        # 最も近い時間を選択
        idx = (temp_subset['Time_ps'] - t_target).abs().idxmin()
        row = temp_subset.loc[idx]
        diff = abs(row['Time_ps'] - t_target)
        
        if diff <= 1.0:
            extracted.append({'Temp': temp, 'MSD': row[msd_col], 'Actual_Time': row['Time_ps']})
            print(f"   - {temp}K: Found at {row['Time_ps']} ps (MSD: {row[msd_col]:.4f})")
        else:
            print(f"   - {temp}K: Skipped (Closest time {row['Time_ps']} ps is too far from {t_target})")
    
    analysis_df = pd.DataFrame(extracted)
    if analysis_df.empty:
        print("   [!] No data extracted for analysis.")
        return pd.DataFrame()

    # 5. 回帰計算
    results = []
    for t_min, t_max in temp_ranges:
        subset = analysis_df[(analysis_df['Temp'] >= t_min) & (analysis_df['Temp'] <= t_max)].dropna()
        
        if len(subset) >= 2:
            x = 1.0 / subset['Temp']
            y = np.log(subset['MSD'])
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
            
            ea_kj = round((-slope * R) / 1000.0, 2)
            results.append({
                'Condition': file_name,
                'Range': f"{t_min}-{t_max}K",
                'Ea [kJ/mol]': ea_kj,
                'R-squared': round(r_value**2, 4),
                'Points': len(subset)
            })
            print(f"5. Calculated Range {t_min}-{t_max}K: Ea={ea_kj}, R2={round(r_value**2, 4)}")
        else:
            print(f"5. Skipped Range {t_min}-{t_max}K: Not enough points ({len(subset)})")
    
    return pd.DataFrame(results)

# --- 実行セクション ---
all_comparison = []
for file in files:
    # パス候補の確認
    input_file = base_path / file / "analysis_results" / "all_trends_summary.csv"
    if not input_file.exists():
        input_file = base_path / file / "analysis_results" / "all_trends.csv"
    
    if not input_file.exists():
        print(f"\n[!] Error: File not found for {file}")
        continue
    
    df = pd.read_csv(input_file)
    res = analyze_ea_variations(df, file)
    
    if not res.empty:
        all_comparison.append(res)

if all_comparison:
    summary_df = pd.concat(all_comparison)
    summary_df.to_csv(output_base_dir / "Ea_Debug_Comparison.csv", index=False)
    print("\n--- Final Results ---")
    print(summary_df)
else:
    print("\n[!] No results were generated. Check the debug messages above.")


--- Debugging Al_Li_F_OH ---
1. Target 'Al_Metal' filtered: 480 rows found.
2. Available Temperatures: [np.float64(300.0), np.float64(400.0), np.float64(500.0), np.float64(600.0), np.float64(750.0), np.float64(900.0)]
3. Using MSD column: 'MSD_Surface'
4. Extracting data near 39.5 ps...
   - 300.0K: Found at 39.493670886075954 ps (MSD: nan)
   - 400.0K: Found at 39.493670886075954 ps (MSD: nan)
   - 500.0K: Found at 39.493670886075954 ps (MSD: nan)
   - 600.0K: Found at 39.493670886075954 ps (MSD: nan)
   - 750.0K: Found at 39.493670886075954 ps (MSD: nan)
   - 900.0K: Found at 39.493670886075954 ps (MSD: nan)
5. Skipped Range 300-400K: Not enough points (0)
5. Skipped Range 300-500K: Not enough points (0)
5. Skipped Range 600-900K: Not enough points (0)
5. Skipped Range 750-900K: Not enough points (0)

[!] No results were generated. Check the debug messages above.


In [11]:
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path

# --- 設定 ---
files = ["Al_Li_F_OH"]
base_path = Path("/home/jovyan/Kaori/MD/LiB_2/structure")
output_base_dir = base_path / "Activation_Energy_Results"
output_base_dir.mkdir(parents=True, exist_ok=True)

R = 8.314
t_target = 39.5
temp_ranges = [(300, 400), (300, 500), (600, 900), (750, 900)]

def analyze_ea_variations(df, file_name, t_target=39.5, target_name='Al_Metal'):
    print(f"\n--- Processing {file_name} ---")
    
    # 1. 前処理：Tempの数値化とTarget絞り込み
    df = df.copy()
    if df['Temp'].dtype == object:
        df['Temp'] = df['Temp'].astype(str).str.replace('K', '', regex=False).astype(float)
    
    # ターゲットで絞り込み、かつ MSD が nan でない行だけを残す
    msd_col = 'MSD_Surface' if 'MSD_Surface' in df.columns else 'MSD_Total'
    df_clean = df[(df['Target'] == target_name) & (df[msd_col].notna())].copy()
    
    if df_clean.empty:
        print(f" [!] No valid data for {target_name} with non-nan MSD.")
        return pd.DataFrame()

    # 2. 39.5psのデータを各温度で抽出
    extracted = []
    for temp in sorted(df_clean['Temp'].unique()):
        temp_subset = df_clean[df_clean['Temp'] == temp]
        
        # 最も target_time に近い行を抽出
        idx = (temp_subset['Time_ps'] - t_target).abs().idxmin()
        row = temp_subset.loc[idx]
        
        if abs(row['Time_ps'] - t_target) <= 1.5: # 許容範囲を少し広めに設定
            extracted.append({'Temp': float(temp), 'MSD': float(row[msd_col])})
            print(f"  - {temp}K: MSD={row[msd_col]:.4f} (at {row['Time_ps']:.2f}ps)")

    analysis_df = pd.DataFrame(extracted)
    if len(analysis_df) < 2:
        print(" [!] Not enough temperature points to perform regression.")
        return pd.DataFrame()

    # 3. 各温度帯で計算
    results = []
    for t_min, t_max in temp_ranges:
        subset = analysis_df[(analysis_df['Temp'] >= t_min) & (analysis_df['Temp'] <= t_max)]
        
        if len(subset) >= 2:
            # log(0) を避けるため微小値を加算または 0 を除外
            subset = subset[subset['MSD'] > 0]
            if len(subset) < 2: continue
            
            x = 1.0 / subset['Temp']
            y = np.log(subset['MSD'])
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
            
            ea_kj = round((-slope * R) / 1000.0, 2)
            results.append({
                'Condition': file_name,
                'Range': f"{t_min}-{t_max}K",
                'Ea [kJ/mol]': ea_kj,
                'R-squared': round(r_value**2, 4),
                'Points': len(subset)
            })
            print(f"  => {t_min}-{t_max}K: Ea={ea_kj} kJ/mol, R2={round(r_value**2, 4)}")
    
    return pd.DataFrame(results)

# --- 実行 ---
all_res = []
for file in files:
    path = base_path / file / "analysis_results" / "all_trends_summary.csv"
    if not path.exists():
        path = base_path / file / "analysis_results" / "all_trends_summary.csv"
    
    if path.exists():
        res = analyze_ea_variations(pd.read_csv(path), file)
        if not res.empty: all_res.append(res)

if all_res:
    pd.concat(all_res).to_csv(output_base_dir / "Ea_Final_Results.csv", index=False)
    print("\n--- Success! Results saved to Activation_Energy_Results/Ea_Final_Results.csv ---")


--- Processing Al_Li_F_OH ---
 [!] No valid data for Al_Metal with non-nan MSD.


In [12]:
import pandas as pd
import numpy as np
from scipy import stats

# データの読み込み
df = pd.read_csv('all_trends_summary (3).csv')

def diagnose_and_calc_ea(df, target_name, t_target=39.5):
    print(f"\n=== Diagnosis for Target: {target_name} ===")
    
    # 対象ターゲットの抽出
    df_target = df[df['Target'] == target_name].copy()
    if df_target.empty:
        print(f" [!] Error: Target '{target_name}' not found in CSV.")
        return

    # MSDカラムの存在と有効チェック
    msd_col = 'MSD_Surface'
    non_null_msd = df_target[msd_col].count()
    print(f" - Total rows: {len(df_target)}")
    print(f" - Valid (non-nan) MSD_Surface values: {non_null_msd}")

    if non_null_msd == 0:
        print(f" [!] Critical: MSD data is missing for {target_name}. Ea calculation is impossible.")
        return

    # 温度の数値化
    if df_target['Temp'].dtype == object:
        df_target['Temp'] = df_target['Temp'].astype(str).str.replace('K', '', regex=False).astype(float)

    # 39.5 ps 時点のデータ抽出
    extracted = []
    for temp in sorted(df_target['Temp'].unique()):
        temp_subset = df_target[(df_target['Temp'] == temp) & (df_target[msd_col].notna())]
        if temp_subset.empty: continue
        
        idx = (temp_subset['Time_ps'] - t_target).abs().idxmin()
        row = temp_subset.loc[idx]
        extracted.append({'Temp': float(temp), 'MSD': float(row[msd_col])})

    # Ea計算
    analysis_df = pd.DataFrame(extracted)
    if len(analysis_df) >= 2:
        # 高温域 (750-900K) のみ
        subset = analysis_df[analysis_df['Temp'] >= 750]
        if len(subset) >= 2:
            slope, _, r_value, _, _ = stats.linregress(1.0/subset['Temp'], np.log(subset['MSD']))
            print(f" -> Ea (750-900K): {round((-slope * 8.314)/1000.0, 2)} kJ/mol (R2={round(r_value**2, 4)})")

# 実行
diagnose_and_calc_ea(df, 'Al_Metal')
diagnose_and_calc_ea(df, 'Al_Oxide')

FileNotFoundError: [Errno 2] No such file or directory: 'all_trends_summary (3).csv'

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ase.io.trajectory import Trajectory
from ase.geometry import get_distances  
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# 全体のフォントサイズ設定
plt.rcParams.update({
    'font.size': 22, 
    'axes.labelsize': 22, 
    'xtick.labelsize': 22, 
    'ytick.labelsize': 22,
    'legend.fontsize': 22
})

def compute_manual_rdf(atoms, indices1, indices2, rmax=6.0, nbins=100):
    if len(indices1) == 0 or len(indices2) == 0:
        return np.zeros(nbins), np.linspace(0, rmax, nbins+1)
    idx1, idx2 = indices1.flatten(), indices2.flatten()
    p1, p2 = atoms.positions[idx1], atoms.positions[idx2]
    _, dists = get_distances(p1, p2, cell=atoms.cell, pbc=atoms.pbc)
    valid_dists = dists[dists > 0.01] if np.array_equal(idx1, idx2) else dists
    hist, bin_edges = np.histogram(valid_dists, range=(0, rmax), bins=nbins)
    return hist, bin_edges

def analyze_md_results(traj_path, output_dir, label, needs_trends=True, needs_rdf=True, needs_density=True):
    print(f"Processing missing items for: {label}")
    try:
        traj = Trajectory(traj_path)
    except Exception as e:
        print(f"Error loading trajectory: {e}"); return

    initial_atoms = traj[0]
    final_atoms = traj[-1]
    cell_vol = final_atoms.get_volume()
    cell_diag = final_atoms.get_cell().diagonal()
    symbols = np.array(final_atoms.get_chemical_symbols())
    indices = {s: np.where(symbols == s)[0] for s in ['Al', 'O', 'Li', 'H', 'F']}

    # 表面/内部 Al分類
    if len(indices['Al']) > 0:
        al_pos = initial_atoms.positions[indices['Al']]
        z_max = np.max(al_pos[:, 2])
        surface_mask = al_pos[:, 2] > (z_max - 4.0)
        idx_al_surface = indices['Al'][surface_mask].flatten()
        idx_al_bulk = indices['Al'][~surface_mask].flatten()
        idx_al_total = indices['Al'].flatten()
    else:
        idx_al_surface = idx_al_bulk = idx_al_total = np.array([])

    # 1. 密度解析 (最終フレームのみで計算可能なため独立)
    if needs_density:
        density_data = {'Z_axis': np.linspace(0, cell_diag[2], 100)}
        bins_z = np.linspace(0, cell_diag[2], 101)
        plt.figure(figsize=(8, 5))
        for spec, idx in indices.items():
            if len(idx.flatten()) > 0:
                hist, _ = np.histogram(final_atoms.positions[idx.flatten(), 2], bins=bins_z)
                density_data[spec] = hist
                plt.plot(density_data['Z_axis'], hist, label=spec, lw=2)
        pd.DataFrame(density_data).to_csv(output_dir / f"{label}_density.csv", index=False)
        plt.title(f"Density: {label}"); plt.legend(); plt.savefig(output_dir / f"{label}_density.png"); plt.close()

    # 2. 時系列解析 (Trends) および RDF (全フレーム走査が必要な項目)
    if needs_trends or needs_rdf:
        time_points, cn_avg_list, reacted_al_list, h2_count_list = [], [], [], []
        msd_total_list, msd_surface_list, msd_bulk_list = [], [], []
        
        rdf_rmax, rdf_nbins = 6.0, 100
        rdf_sum = np.zeros(rdf_nbins)
        rdf_count = 0
        start_rdf_frame = int(len(traj) * 0.5)
        idx_reactants = np.concatenate([indices['O'].flatten(), indices['F'].flatten()])

        initial_pos_total = initial_atoms.positions[idx_al_total] if len(idx_al_total) > 0 else None
        initial_pos_surface = initial_atoms.positions[idx_al_surface] if len(idx_al_surface) > 0 else None
        initial_pos_bulk = initial_atoms.positions[idx_al_bulk] if len(idx_al_bulk) > 0 else None
        dt_ps = 40.0 / max(len(traj)-1, 1)

        for i, atoms in enumerate(traj):
            time_points.append(i * dt_ps)
            if needs_trends:
                # 配位数/反応数
                if len(idx_al_total) > 0 and len(idx_reactants) > 0:
                    _, d_len = get_distances(atoms.positions[idx_al_total], atoms.positions[idx_reactants], cell=atoms.cell, pbc=atoms.pbc)
                    cn_per_atom = np.sum(d_len.reshape(len(idx_al_total), len(idx_reactants)) < 2.5, axis=1)
                    cn_avg_list.append(np.mean(cn_per_atom))
                    reacted_al_list.append(np.sum(cn_per_atom >= 1))
                else:
                    cn_avg_list.append(0); reacted_al_list.append(0)
                
                # MSD
                def get_msd(pos, init): return np.mean(np.sum((pos - init)**2, axis=1)) if init is not None else 0
                msd_total_list.append(get_msd(atoms.positions[idx_al_total], initial_pos_total))
                msd_surface_list.append(get_msd(atoms.positions[idx_al_surface], initial_pos_surface))
                msd_bulk_list.append(get_msd(atoms.positions[idx_al_bulk], initial_pos_bulk))

                # H2検出
                idx_h = indices['H'].flatten()
                if len(idx_h) > 1:
                    _, h_d = get_distances(atoms.positions[idx_h], atoms.positions[idx_h], cell=atoms.cell, pbc=atoms.pbc)
                    h2_count_list.append(np.sum(h_d[h_d > 0.01] < 0.85) // 2)
                else:
                    h2_count_list.append(0)

            if needs_rdf and i >= start_rdf_frame:
                hist, edges = compute_manual_rdf(atoms, indices['Al'], idx_reactants, rmax=rdf_rmax, nbins=rdf_nbins)
                rdf_sum += hist; rdf_count += 1; bin_edges = edges

        if needs_trends:
            df_trends = pd.DataFrame({'Time_ps': time_points, 'CN_Avg': cn_avg_list, 'Reacted_Al_Count': reacted_al_list, 
                                      'H2_Count': h2_count_list, 'MSD_Total': msd_total_list, 'MSD_Surface': msd_surface_list, 'MSD_Bulk': msd_bulk_list})
            df_trends.to_csv(output_dir / f"{label}_trends.csv", index=False)
            # グラフ描画
            plt.figure(); plt.plot(time_points, msd_surface_list, label='Surface'); plt.plot(time_points, msd_bulk_list, label='Bulk')
            plt.title(f"MSD: {label}"); plt.legend(); plt.savefig(output_dir / f"{label}_msd_detailed.png"); plt.close()

        if needs_rdf and rdf_count > 0:
            r = 0.5 * (bin_edges[1:] + bin_edges[:-1])
            rho = len(idx_reactants) / cell_vol
            gr = (rdf_sum / rdf_count) / (len(indices['Al']) * (4 * np.pi * r**2 * (bin_edges[1]-bin_edges[0])) * rho)
            pd.DataFrame({'r': r, 'g_r': gr}).to_csv(output_dir / f"{label}_rdf_Al_O.csv", index=False)

def main_analysis():
    files = ["Al_LiOH_v8", "Al_Li_F_OH", "Al_Li_F", "Al_Na_F_OH"]
    required_trends_cols = ['Time_ps', 'CN_Avg', 'Reacted_Al_Count', 'H2_Count', 'MSD_Total', 'MSD_Surface', 'MSD_Bulk']
    
    for file in files:
        base_dir = Path(f"/home/jovyan/Kaori/MD/LiB_2/structure/{file}")
        analysis_dir = base_dir / "analysis_results"
        analysis_dir.mkdir(parents=True, exist_ok=True)
        
        for target in ["Al_Metal", "Al_Oxide"]:
            for temp in [300, 400, 500, 600, 750, 900]:
                label = f"{target}_{temp}K"
                needs_trends, needs_rdf, needs_density = True, True, True
                
                # --- 項目ごとのスキップ判定 ---
                # 1. Trends CSVの項目チェック
                trends_path = analysis_dir / f"{label}_trends.csv"
                if trends_path.exists():
                    try:
                        df_tmp = pd.read_csv(trends_path)
                        if all(col in df_tmp.columns for col in required_trends_cols):
                            needs_trends = False
                    except: pass
                
                # 2. RDF/Densityのファイル存在チェック
                if (analysis_dir / f"{label}_rdf_Al_O.csv").exists(): needs_rdf = False
                if (analysis_dir / f"{label}_density.csv").exists(): needs_density = False

                if not (needs_trends or needs_rdf or needs_density):
                    print(f"Skip: {label} in {file} (All items exist)")
                    continue

                traj_file = base_dir / f"{target}_md_{temp}K.traj"
                if traj_file.exists():
                    analyze_md_results(traj_file, analysis_dir, label, needs_trends, needs_rdf, needs_density)

if __name__ == "__main__":
    main_analysis()

Skip: Al_Metal_300K in Al_LiOH_v8 (All items exist)
Skip: Al_Metal_400K in Al_LiOH_v8 (All items exist)
Skip: Al_Metal_500K in Al_LiOH_v8 (All items exist)
Skip: Al_Oxide_300K in Al_LiOH_v8 (All items exist)
Skip: Al_Oxide_400K in Al_LiOH_v8 (All items exist)
Skip: Al_Oxide_500K in Al_LiOH_v8 (All items exist)
Processing missing items for: Al_Metal_300K


KeyboardInterrupt: 